In [ ]:
!pip install unidecode
!pip install fuzzywuzzy
!pip install Levenshtein

In [ ]:
# Imports generales

import pandas as pd
import json
import io
from google.colab import files, drive
import matplotlib.pyplot as plt
import plotly.express as px
import re
import numpy as np
import unidecode
from collections import defaultdict
from fuzzywuzzy import fuzz, process

regex_pat = re.compile(r'FUZ', flags=re.IGNORECASE)

In [ ]:
drive.mount('/content/drive/')
base_folder_string = '/content/drive/MyDrive/Colab Notebooks/TFM//'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


---
# 0. Definición de funciones de útilidad
---

In [ ]:
def single_player_row(dataframe, column_key):
    if dataframe.shape[0] == 1:
        return dataframe
    else:
        row = dataframe[dataframe[column_key] == "TOT"]
        row[column_key] = dataframe.iloc[-1,:][column_key]
        return row

In [ ]:
def initial_name_cleaning(dataframe):
  clean_df = dataframe.copy()
  clean_df['player'] = clean_df['player'].apply(remove_accents)
  clean_df['player'] = clean_df['player'].str.replace('Jr.', '', regex=False)
  clean_df['player'] = clean_df['player'].str.replace('II', '', regex=False)
  clean_df['player'] = clean_df['player'].str.replace('III', '', regex=False)
  clean_df['player'] = clean_df['player'].str.replace('.', '', regex=False)
  clean_df['player'] = clean_df['player'].str.replace('*', '', regex=False)

  print(dataframe.shape)

  return clean_df

In [ ]:
# Función para importar dataframes de BPM

def get_bpm_dataframes(seasons_list, string):
    dataframes = []
    joined_dataframes = []

    for season in seasons_list:

        df = pd.read_csv(string.format(season))
        # print(df.columns.values)
        df = df.apply(pd.to_numeric, errors='ignore')
        df = df.groupby(["Player"], group_keys=False).apply(single_player_row, column_key='Tm')
        df["season"] = season # Función para agregar columna de Temporada, se usa para poder generar el df grande pero no se usa para clustering
        dataframes.append(df)

    return dataframes

In [ ]:
def convert_season_to_int(season):
    # Dividir string usando .split
    parts = season.split('-')

    # Convertir el número restante a entero y sumar 2000 ej: 14 + 2000 = 2014
    return int(parts[0]) + 1

In [ ]:
def remove_accents(a):
    return unidecode.unidecode(a)

In [ ]:
def choose_standard_name(names):
    if len(names) == 1:
        return names[0]

    max_similarity = -1
    standard_name = None

    for name in names:
        similarity_sum = sum(fuzz.ratio(name, other_name) for other_name in names if other_name != name)
        if similarity_sum > max_similarity:
            max_similarity = similarity_sum
            standard_name = name

    return standard_name

In [ ]:
def standardize_player_name(name, player_dict):
    for key, values in player_dict.items():
        if name in values:
            return key  # Return the standardized name if found in the dictionary
    return name  # Return the original name if not found

In [ ]:
def get_dataframes(seasons_list, str_list):
    dataframes = []
    joined_dataframes = []

    for csv_string in str_list:
      for season in seasons_list:

          df = pd.read_csv(csv_string.format(season))

          columns_to_drop = [col for col in df.columns if 'RANK' in col]
          df.drop(columns=columns_to_drop, inplace=True)
          df = df.dropna(axis=1, how='all')
          df["season"] = season # Función para agregar columna de Temporada, se usa para poder generar el df grande pero no se usa para clustering
          dataframes.append(df)

    return dataframes

In [ ]:
def concatenate_dataframes(dataframes):
    # Create a dictionary to group dataframes by columns
    grouped_dataframes = {}

    for df in dataframes:
        # Create a hashable representation of the DataFrame's columns
        df.columns = df.columns.str.lower()
        columns_key = tuple(sorted(df.columns))

        if columns_key in grouped_dataframes:
            # If columns match, append the DataFrame to the existing list
            grouped_dataframes[columns_key].append(df)
        else:
            # If columns don't match, create a new list for this set of columns
            grouped_dataframes[columns_key] = [df]

    concatenated_dataframes = []

    for key, df_list in grouped_dataframes.items():
        # Concatenate DataFrames with the same columns
        concatenated_df = pd.concat(df_list, ignore_index=True)
        concatenated_dataframes.append(concatenated_df)

    return concatenated_dataframes

In [ ]:
def name_dataframes(dataframes, names_ls):
  renamed_dataframes = []

  for added_str, df in zip(names_ls, dataframes):
    df.columns = [str(col) + added_str for col in df.columns]
    renamed_dataframes.append(df)
    print(added_str)

  return renamed_dataframes

In [ ]:
def keep_join_columns(df, name_ls):
  # Iterate through the column names and modify them
  for col in df.columns:
      for name in name_ls:
          if name in col:
              df.rename(columns={col: name}, inplace=True)

  # Now the DataFrame has columns renamed as per your requirement
  return df

In [ ]:
# Función para encontrar duplicados en las columnas:

def find_duplicate_columns(df):
    duplicate_columns = set()
    columns = df.columns.tolist()

    for i in range(len(columns)):
        for j in range(i + 1, len(columns)):
            col1 = df[columns[i]]
            col2 = df[columns[j]]

            if col1.equals(col2):
                duplicate_columns.add(columns[j])

    return list(duplicate_columns)

---
# 1. Importar csv y separar DataFrames de clustering Temporada Regular
---

In [ ]:
# Estadísticas disponibles desde 2014
regseas_cnshoot_string = '/content/drive/MyDrive/Colab Notebooks/TFM/track_catch_and_shoot/regseas_catchandshoot_pergame_{}.csv'
regseas_defimpact_string = '/content/drive/MyDrive/Colab Notebooks/TFM/track_def_impact/regseas_def_impact_pergame_{}.csv'
regseas_defreb_string = '/content/drive/MyDrive/Colab Notebooks/TFM/track_def_reb/regseas_defreb_pergame_{}.csv'
regseas_drives_string = '/content/drive/MyDrive/Colab Notebooks/TFM/track_drives/regseas_drives_pergame_{}.csv'
regseas_offreb_string = '/content/drive/MyDrive/Colab Notebooks/TFM/track_off_reb/regseas_offreb_pergame_{}.csv'
regseas_pass_string = '/content/drive/MyDrive/Colab Notebooks/TFM/track_passing/regseas_passing_pergame_{}.csv'
regseas_pullup_string = '/content/drive/MyDrive/Colab Notebooks/TFM/track_pull_up_shoot/regseas_pullup_pergame_{}.csv'
regseas_reb_string = '/content/drive/MyDrive/Colab Notebooks/TFM/track_rebounding/regseason_rebounding_pergame_{}.csv'
regseas_shooteff_string = '/content/drive/MyDrive/Colab Notebooks/TFM/track_shoot_eff/regseas_shoot_eff_pergame_{}.csv'
regseas_speed_string = '/content/drive/MyDrive/Colab Notebooks/TFM/track_speed_and_distance/regseas_speed_pergame_{}.csv'

# Estadísticas disponibles desde 1997
# NBA
regseas_adv_string = '/content/drive/MyDrive/Colab Notebooks/TFM/advanced/regseas_adv_per100_{}.csv'
regseas_def_string = '/content/drive/MyDrive/Colab Notebooks/TFM/defense/regseas_defense_per100_{}.csv'
regseas_misc_string = '/content/drive/MyDrive/Colab Notebooks/TFM/miscellaneous/regseas_misc_per100_{}.csv'
regseas_scoring_string = '/content/drive/MyDrive/Colab Notebooks/TFM/scoring/regseas_scoring_per100_{}.csv'
regseas_trad_string = '/content/drive/MyDrive/Colab Notebooks/TFM/traditional/regseas_trad_per100_{}.csv'
regseas_shootzone_string = '/content/drive/MyDrive/Colab Notebooks/TFM/shooting_by_zone/regseas_shootzone_pergame_{}.csv'

# BBALLREF
pbp_bbref_string = '/content/drive/MyDrive/Colab Notebooks/TFM/pbp_data_bballref/reg_season_{}.csv'
shoot_bbref_string = '/content/drive/MyDrive/Colab Notebooks/TFM/shooting_data_bballref/reg_season_{}.csv'
bpm_bbref_string_rs = '/content/drive/MyDrive/Colab Notebooks/TFM/rs_bpm_data_bballref/reg_season_{}.csv'
# BBALLREF Playoffs
bpm_bbref_string_po = '/content/drive/MyDrive/Colab Notebooks/TFM/playoff_bpm_data_bballref/playoffs_{}.csv'

In [ ]:
# Definición listas de estadísticas y prefijos:

csv_str_list = [regseas_adv_string, regseas_def_string, regseas_misc_string, regseas_scoring_string, \
                regseas_shootzone_string, regseas_cnshoot_string, regseas_defimpact_string, regseas_defreb_string, \
                regseas_drives_string, regseas_offreb_string, regseas_pass_string, regseas_pullup_string, regseas_reb_string, \
                regseas_shooteff_string, regseas_speed_string, regseas_trad_string]

def_list = [regseas_speed_string, regseas_def_string, regseas_misc_string, regseas_defimpact_string, regseas_defreb_string, regseas_adv_string]

def_ls_names = ['_speed','_def', '_misc', '_defim', '_dreb', '_adv']

off_list = [regseas_adv_string, regseas_misc_string, regseas_scoring_string, \
                regseas_shootzone_string, regseas_cnshoot_string, \
                regseas_drives_string, regseas_offreb_string, regseas_pass_string, regseas_pullup_string, \
                regseas_shooteff_string, regseas_speed_string, regseas_trad_string]

off_ls_names = ['_adv', '_misc', '_scor', '_szone', '_cns', '_dvs', '_orb', '_pass', '_pup', '_seff', '_speed', '_trad']

In [ ]:
# Definición lista de temporadas:

seasons_list = ['2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23']

In [ ]:
# Importación de dataframes OFENSIVO y DEFENSIVO

rs_def_df = get_dataframes(seasons_list, def_list)
rs_off_df = get_dataframes(seasons_list, off_list)

Importación con temporadas desde 1997

In [ ]:
def_list_ext_nba = [regseas_def_string, regseas_misc_string, regseas_adv_string]
def_list_ext_bbref = [bpm_bbref_string_rs, pbp_bbref_string]

def_ls_names_ext = ['_def', '_misc', '_adv']
def_ls_bbref = ['_bpm', '_pbp']


off_list_ext_nba = [regseas_adv_string, regseas_misc_string, regseas_scoring_string, regseas_shootzone_string, regseas_trad_string]
off_list_ext_bbref = [bpm_bbref_string_rs, pbp_bbref_string, shoot_bbref_string]

off_ls_names_ext = ['_adv', '_misc', '_scor', '_szone', '_trad']
off_ls_bbref = ['_bpm', '_pbp', '_shoot']

In [ ]:
increased_season_ls = []

for year in range(1996, 2023):
    season = f"{year}-{str(year + 1)[-2:]}"
    increased_season_ls.append(season)

print(increased_season_ls[0:3])

['1996-97', '1997-98', '1998-99']


In [ ]:
# Importación de dataframes OFENSIVO y DEFENSIVO CON MAS TEMPORADAS

rs_def_extended = get_dataframes(increased_season_ls, def_list_ext_nba)
rs_off_extended = get_dataframes(increased_season_ls, off_list_ext_nba)

In [ ]:
increased_season_bpm = []

for year in range(1997, 2024):
    increased_season_bpm.append(year)

print(increased_season_bpm[0], '-', increased_season_bpm[26])

1997 - 2023


In [ ]:
rs_bbref_def = get_dataframes(increased_season_bpm, def_list_ext_bbref)
rs_bbref_off = get_dataframes(increased_season_bpm, off_list_ext_bbref)

<ipython-input-175-aaf8ca795482>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["season"] = season # Función para agregar columna de Temporada, se usa para poder generar el df grande pero no se usa para clustering


In [ ]:
def update_column_names(df, new_column_names):

    df.columns = new_column_names

    return df

In [ ]:
new_column_names_pbp = ['Rk', 'Player', 'position', 'age', 'Tm', 'g', 'minutes_played',
                        'pg%', 'sg%', 'sf%', 'pf%', 'c%', '+/-_oncourt', '+/-_on-off',
                        'to_badpass', 'to_lostball', 'fouls_committed_shoot', 'fouls_committed_off',
                        'fouls_drawn_shoot', 'fouls_drawn_off', 'pga', 'and1', 'blocked', 'season']

new_column_names_shoot = ['Rk', 'Player', 'position', 'age', 'Tm', 'g', 'minutes_played', 'fg%',
                          'dist', 'unnamed_9_level_1', '%_fga_by_distance_2p', '%_fga_by_distance_0-3',
                          '%_fga_by_distance_3-10', '%_fga_by_distance_10-16', '%_fga_by_distance_16-3p',
                          '%_fga_by_distance_3p', 'unnamed_16_level_1', 'fg%_2p', 'fg%_0-3', 'fg%_3-10',
                          'fg%_10-16', 'fg%_16-3p', 'fg%_3p', 'unnamed_23_level_1', '%_fg_astd_2p',
                          '%_fg_astd_3p', 'unnamed_26_level_1', 'dunks_%fga', 'dunks_#', 'unnamed_29_level_1',
                          'corner_3s_%3pa', 'corner_3s_3p%', 'unnamed_32_level_1', 'heaves_att', 'heaves_#',
                          'season']

In [ ]:
for i, df in enumerate(rs_bbref_def[27:]):
  df = update_column_names(df, new_column_names_pbp)

In [ ]:
for i, df in enumerate(rs_bbref_off[27:54]):
  df = update_column_names(df, new_column_names_pbp)

for i, df in enumerate(rs_bbref_off[54:]):
  df = update_column_names(df, new_column_names_shoot)

In [ ]:
for i, df in enumerate(rs_bbref_def):
  df = df[df['Rk'] != 'Rk']
  df = df.groupby(["Player"], group_keys=False).apply(single_player_row, column_key='Tm')
  rs_bbref_def[i] = df  # Reassign the modified DataFrame back to the collection


for i, df in enumerate(rs_bbref_off):
  df = df[df['Rk'] != 'Rk']
  df = df.groupby(["Player"], group_keys=False).apply(single_player_row, column_key='Tm')
  rs_bbref_off[i] = df  # Reassign the modified DataFrame back to the collection

---
## 1.1. Filtrado de columnas para nuevas características
---

DEFENSIVO:

In [ ]:
# Borrado de jugadores con menos de 10 partidos
newfeature_rs_def = []

for dataframe in rs_def_df:
  dataframe = dataframe[dataframe["GP"] > 10]
  dataframe.columns = dataframe.columns.str.lower()
  dataframe = dataframe.sort_values(by='player')
  newfeature_rs_def.append(dataframe)

In [ ]:
len(newfeature_rs_def)

60

In [ ]:
newfeature_rs_def[9].head(2)

,player,team,gp,w,l,min,dist. feet,dist. miles,dist. miles off,dist. miles def,avg speed,avg speed off,avg speed def,season
0,A.J. Lawson,DAL,15,5,10,7.2,3155.2,0.60,0.32,0.28,4.64,4.88,4.30,2022-23
1,AJ Green,MIL,35,27,8,9.9,4053.2,0.77,0.42,0.34,4.35,4.73,3.96,2022-23


In [ ]:
speed_str = '_speed'
keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
keep_col_speed = ['player', 'team', 'gp', 'min', 'dist.\xa0miles\xa0def', 'season']

for i in range(10):
    df = newfeature_rs_def[i]

    selected_cols = [col for col in keep_col_speed if col not in keep_col_exceptions]
    df = df[keep_col_speed]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_def[i] = df

In [ ]:
newfeature_rs_def[10].columns.values

array(['unnamed: 0', 'player', 'team', 'age', 'gp', 'w', 'l', 'min',
       'def\xa0rtg', 'dreb', 'dreb%', '%dreb', 'stl', 'stl%', 'blk',
       '%blk', 'opp\xa0ptsoff\xa0tov', 'opp\xa0pts2nd\xa0chance',
       'opp\xa0ptsfb', 'opp\xa0ptspaint', 'defws', 'season'], dtype=object)

In [ ]:
keep_col_def = ['player', 'team', 'gp', 'min', 'def\xa0rtg', 'dreb', 'dreb%', '%dreb', 'stl', 'stl%', 'blk',
                '%blk', 'opp\xa0ptsoff\xa0tov', 'opp\xa0pts2nd\xa0chance', 'opp\xa0ptsfb', 'opp\xa0ptspaint', 'defws', 'season']

keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
speed_str = '_def'

for i in range(10, 20):  # Using range to cover indices 10 to 19
    df = newfeature_rs_def[i]

    # Select columns from keep_col_def and then rename columns
    selected_cols = [col for col in keep_col_def if col not in keep_col_exceptions]
    df = df[keep_col_def]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_def[i] = df

In [ ]:
newfeature_rs_def[29].columns.values

array(['unnamed: 0', 'player', 'team', 'age', 'gp', 'w', 'l', 'min',
       'ptsoff\xa0to', '2ndpts', 'fbps', 'pitp', 'opp\xa0ptsoff\xa0to',
       'opp2nd\xa0pts', 'oppfbps', 'opppitp', 'blk', 'blka', 'pf', 'pfd',
       'season'], dtype=object)

In [ ]:
keep_col_misc = ['player', 'team', 'gp', 'min', 'opp\xa0ptsoff\xa0to', 'opp2nd\xa0pts', 'oppfbps', 'opppitp', 'blk', 'blka', 'pf', 'season']

keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
speed_str = '_misc'

for i in range(20, 30):
    df = newfeature_rs_def[i]

    selected_cols = [col for col in keep_col_misc if col not in keep_col_exceptions]
    df = df[keep_col_misc]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_def[i] = df

In [ ]:
newfeature_rs_def[39].columns.values

array(['player', 'team', 'gp', 'min', 'w', 'l', 'stl', 'blk', 'dreb',
       'dfgm', 'dfga', 'dfg%', 'season'], dtype=object)

In [ ]:
keep_col_def_imp = ['player', 'team', 'gp', 'min','dfgm', 'dfga', 'dfg%', 'season']

keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
speed_str = '_imp'

for i in range(30, 40):
    df = newfeature_rs_def[i]

    selected_cols = [col for col in keep_col_def_imp if col not in keep_col_exceptions]
    df = df[keep_col_def_imp]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_def[i] = df

In [ ]:
newfeature_rs_def[49].columns.values

array(['player', 'team', 'gp', 'w', 'l', 'min', 'dreb', 'contesteddreb',
       'contesteddreb%', 'drebchances', 'drebchance%',
       'deferreddreb\xa0chances', 'adjusteddreb\xa0chance%',
       'avg\xa0drebdistance', 'season'], dtype=object)

In [ ]:
keep_col_def_reb = ['player', 'team', 'gp', 'min', 'dreb', 'contesteddreb',
       'contesteddreb%', 'drebchances', 'drebchance%',
       'deferreddreb\xa0chances', 'adjusteddreb\xa0chance%',
       'avg\xa0drebdistance', 'season']

keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
speed_str = '_reb'

for i in range(40, 50):
    df = newfeature_rs_def[i]

    selected_cols = [col for col in keep_col_def_reb if col not in keep_col_exceptions]
    df = df[keep_col_def_reb]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_def[i] = df

In [ ]:
newfeature_rs_def[59].columns.values

array(['unnamed: 0', 'player', 'team', 'age', 'gp', 'w', 'l', 'min',
       'offrtg', 'defrtg', 'netrtg', 'ast%', 'ast/to', 'ast\xa0ratio',
       'oreb%', 'dreb%', 'reb%', 'to\xa0ratio', 'efg%', 'ts%', 'usg%',
       'pace', 'pie', 'poss', 'season'], dtype=object)

In [ ]:
keep_col_defadv = ['player', 'team', 'gp', 'min',
       'defrtg', 'dreb%', 'pie', 'poss', 'season']

keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
speed_str = '_defadv'

for i in range(50, len(newfeature_rs_def)):
    df = newfeature_rs_def[i]

    selected_cols = [col for col in keep_col_defadv if col not in keep_col_exceptions]
    df = df[keep_col_defadv]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_def[i] = df

OFENSIVO:

In [ ]:
# Borrado de jugadores con menos de 10 partidos
newfeature_rs_off = []

for dataframe in rs_off_df:
  dataframe.columns = dataframe.columns.str.lower()
  dataframe = dataframe.sort_values(by='player')
  newfeature_rs_off.append(dataframe)

In [ ]:
len(newfeature_rs_off)

120

In [ ]:
newfeature_rs_off[9].columns.values

array(['unnamed: 0', 'player', 'team', 'age', 'gp', 'w', 'l', 'min',
       'offrtg', 'defrtg', 'netrtg', 'ast%', 'ast/to', 'ast\xa0ratio',
       'oreb%', 'dreb%', 'reb%', 'to\xa0ratio', 'efg%', 'ts%', 'usg%',
       'pace', 'pie', 'poss', 'season'], dtype=object)

In [ ]:
keep_col_adv = ['player', 'team', 'gp', 'min',
       'offrtg', 'ast%', 'ast/to', 'ast\xa0ratio',
       'oreb%', 'to\xa0ratio', 'ts%', 'usg%',
       'pace', 'pie', 'poss', 'season']

keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
speed_str = '_adv'

for i in range(0, 10):
    df = newfeature_rs_off[i]

    selected_cols = [col for col in keep_col_adv if col not in keep_col_exceptions]
    df = df[keep_col_adv]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_off[i] = df

In [ ]:
newfeature_rs_off[19].columns.values

array(['unnamed: 0', 'player', 'team', 'age', 'gp', 'w', 'l', 'min',
       'ptsoff\xa0to', '2ndpts', 'fbps', 'pitp', 'opp\xa0ptsoff\xa0to',
       'opp2nd\xa0pts', 'oppfbps', 'opppitp', 'blk', 'blka', 'pf', 'pfd',
       'season'], dtype=object)

In [ ]:
keep_col_misc = ['player', 'team', 'gp', 'min',
       'ptsoff\xa0to', '2ndpts', 'fbps', 'pitp', 'pfd',
       'season']

keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
speed_str = '_misc'

for i in range(10, 20):
    df = newfeature_rs_off[i]

    selected_cols = [col for col in keep_col_misc if col not in keep_col_exceptions]
    df = df[keep_col_misc]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_off[i] = df

In [ ]:
newfeature_rs_off[29].columns.values

array(['unnamed: 0', 'player', 'team', 'age', 'gp', 'w', 'l', 'min',
       '%fga2pt', '%fga3pt', '%pts2pt', '%pts2pt\xa0mr', '%pts3pt',
       '%ptsfbps', '%ptsft', '%ptsoffto', '%ptspitp', '2fgm%ast',
       '2fgm%uast', '3fgm%ast', '3fgm%uast', 'fgm%ast', 'fgm%uast',
       'season'], dtype=object)

In [ ]:
keep_col_scoring = ['player', 'team', 'age', 'min',
       '%fga2pt', '%fga3pt', '%pts2pt', '%pts2pt\xa0mr', '%pts3pt',
       '%ptsfbps', '%ptsft', '%ptsoffto', '%ptspitp', '2fgm%ast',
       '2fgm%uast', '3fgm%ast', '3fgm%uast', 'fgm%ast', 'fgm%uast',
       'season']

keep_col_exceptions = ['player', 'team', 'age', 'min', 'season']
speed_str = '_scoring'

for i in range(20, 30):
    df = newfeature_rs_off[i]

    selected_cols = [col for col in keep_col_scoring if col not in keep_col_exceptions]
    df = df[keep_col_scoring]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_off[i] = df

In [ ]:
newfeature_rs_off[39].columns.values

array(['player', 'team', 'age', 'ra_fgm', 'ra_fga', 'ra_fg%', 'paint_fgm',
       'paint_fga', 'paint_fg%', 'mr_fgm', 'mr_fga', 'mr_fg%', 'lc3_fgm',
       'lc3_fga', 'lc3_fg%', 'rc3_fgm', 'rc3_fga', 'rc3_fg%', 'c3_fga',
       'c3_fga.1', 'c3_fga.2', 'ab3_fgm', 'ab3_fga', 'ab3_fg%', 'season'],
      dtype=object)

In [ ]:
keep_col_shootingzone = ['player', 'team', 'ra_fgm', 'ra_fga', 'ra_fg%', 'paint_fgm',
       'paint_fga', 'paint_fg%', 'mr_fgm', 'mr_fga', 'mr_fg%', 'lc3_fgm',
       'lc3_fga', 'lc3_fg%', 'rc3_fgm', 'rc3_fga', 'rc3_fg%', 'c3_fga',
       'c3_fga.1', 'c3_fga.2', 'ab3_fgm', 'ab3_fga', 'ab3_fg%', 'season']

keep_col_exceptions = ['player', 'team', 'season']
speed_str = '_shootingzone'

for i in range(30, 40):
    df = newfeature_rs_off[i]

    selected_cols = [col for col in keep_col_shootingzone if col not in keep_col_exceptions]
    df = df[keep_col_shootingzone]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_off[i] = df

In [ ]:
newfeature_rs_off[49].columns.values

array(['player', 'team', 'gp', 'min', 'pts', 'fgm', 'fga', 'fg%', '3pm',
       '3pa', '3p%', 'efg%', 'season'], dtype=object)

In [ ]:
keep_col_cnshoot = ['player', 'team', 'gp', 'min', 'pts', 'fgm', 'fga', 'fg%', '3pm',
       '3pa', '3p%', 'efg%', 'season']

keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
speed_str = '_catch&shoot'

for i in range(40, 50):
    df = newfeature_rs_off[i]

    selected_cols = [col for col in keep_col_cnshoot if col not in keep_col_exceptions]
    df = df[keep_col_cnshoot]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_off[i] = df

In [ ]:
newfeature_rs_off[59].columns.values

array(['player', 'team', 'gp', 'w', 'l', 'min', 'drives', 'fgm', 'fga',
       'fg%', 'ftm', 'fta', 'ft%', 'pts', 'pts%', 'pass', 'pass%', 'ast',
       'ast%', 'to', 'tov%', 'pf', 'pf%', 'season'], dtype=object)

In [ ]:
keep_col_drives = ['player', 'team', 'gp', 'min', 'drives', 'fgm', 'fga',
       'fg%', 'ftm', 'fta', 'ft%', 'pts', 'pts%', 'pass', 'pass%', 'ast',
       'ast%', 'to', 'tov%', 'pf', 'pf%', 'season']

keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
speed_str = '_drives'

for i in range(50, 60):
    df = newfeature_rs_off[i]

    selected_cols = [col for col in keep_col_drives if col not in keep_col_exceptions]
    df = df[keep_col_drives]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_off[i] = df

In [ ]:
newfeature_rs_off[69].columns.values

array(['player', 'team', 'gp', 'w', 'l', 'min', 'oreb', 'contestedoreb',
       'contestedoreb%', 'orebchances', 'orebchance%',
       'deferredoreb\xa0chances', 'adjustedoreb\xa0chance%',
       'avg\xa0orebdistance', 'season'], dtype=object)

In [ ]:
keep_col_oreb = ['player', 'team', 'gp', 'min', 'oreb', 'contestedoreb',
       'contestedoreb%', 'orebchances', 'orebchance%',
       'deferredoreb\xa0chances', 'adjustedoreb\xa0chance%',
       'avg\xa0orebdistance', 'season']

keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
speed_str = '_oreb'

for i in range(60, 70):
    df = newfeature_rs_off[i]

    selected_cols = [col for col in keep_col_oreb if col not in keep_col_exceptions]
    df = df[keep_col_oreb]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_off[i] = df

In [ ]:
newfeature_rs_off[79].columns.values

array(['player', 'team', 'gp', 'w', 'l', 'min', 'passesmade',
       'passesreceived', 'ast', 'secondaryast', 'potentialast',
       'ast\xa0ptscreated', 'ast\xa0ptscreated.1', 'astadj',
       'ast\xa0topass%', 'season'], dtype=object)

In [ ]:
keep_col_passing = ['player', 'team', 'gp', 'min', 'passesmade',
       'passesreceived', 'ast', 'secondaryast', 'potentialast',
       'ast\xa0ptscreated', 'astadj',
       'ast\xa0topass%', 'season']

keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
speed_str = '_passing'

for i in range(70, 80):
    df = newfeature_rs_off[i]

    selected_cols = [col for col in keep_col_passing if col not in keep_col_exceptions]
    df = df[keep_col_passing]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_off[i] = df

In [ ]:
newfeature_rs_off[89].columns.values

array(['player', 'team', 'gp', 'w', 'l', 'min', 'pts', 'fgm', 'fga',
       'fg%', '3pm', '3pa', '3p%', 'efg%', 'season'], dtype=object)

In [ ]:
keep_col_pullup = ['player', 'team', 'gp', 'min', 'pts', 'fgm', 'fga',
       'fg%', '3pm', '3pa', '3p%', 'season']

keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
speed_str = '_pullup'

for i in range(80, 90):
    df = newfeature_rs_off[i]

    selected_cols = [col for col in keep_col_pullup if col not in keep_col_exceptions]
    df = df[keep_col_pullup]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_off[i] = df

In [ ]:
newfeature_rs_off[99].columns.values

array(['player', 'team', 'gp', 'w', 'l', 'min', 'pts', 'drivepts',
       'drivefg%', 'c&spts', 'c&sfg%', 'pull\xa0uppts', 'pull\xa0upfg%',
       'painttouch\xa0pts', 'painttouch\xa0fg%', 'posttouch\xa0pts',
       'posttouch\xa0fg%', 'elbowtouch\xa0pts', 'elbowtouch\xa0fg%',
       'efg%', 'season'], dtype=object)

In [ ]:
keep_col_shooting_eff = ['player', 'team', 'gp', 'min', 'drivepts',
       'drivefg%', 'c&spts', 'c&sfg%', 'pull\xa0uppts', 'pull\xa0upfg%',
       'painttouch\xa0pts', 'painttouch\xa0fg%', 'posttouch\xa0pts',
       'posttouch\xa0fg%', 'elbowtouch\xa0pts', 'elbowtouch\xa0fg%', 'season']

keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
speed_str = '_shooting_eff'

for i in range(90, 100):
    df = newfeature_rs_off[i]

    selected_cols = [col for col in keep_col_shooting_eff if col not in keep_col_exceptions]
    df = df[keep_col_shooting_eff]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_off[i] = df

In [ ]:
newfeature_rs_off[109].columns.values

array(['player', 'team', 'gp', 'w', 'l', 'min', 'dist.\xa0feet',
       'dist.\xa0miles', 'dist.\xa0miles\xa0off', 'dist.\xa0miles\xa0def',
       'avg\xa0speed', 'avg\xa0speed\xa0off', 'avg\xa0speed\xa0def',
       'season'], dtype=object)

In [ ]:
keep_col_speed = ['player', 'team', 'gp', 'min', 'dist.\xa0miles\xa0off',
       'avg\xa0speed\xa0off',
       'season']

keep_col_exceptions = ['player', 'team', 'gp', 'min', 'season']
speed_str = '_speed'

for i in range(100, 110):
    df = newfeature_rs_off[i]

    selected_cols = [col for col in keep_col_speed if col not in keep_col_exceptions]
    df = df[keep_col_speed]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_off[i] = df

In [ ]:
newfeature_rs_off[119].columns.values

array(['unnamed: 0', 'player', 'team', 'age', 'gp', 'w', 'l', 'min',
       'pts', 'fgm', 'fga', 'fg%', '3pm', '3pa', '3p%', 'ftm', 'fta',
       'ft%', 'oreb', 'dreb', 'reb', 'ast', 'tov', 'stl', 'blk', 'pf',
       'fp', 'dd2', 'td3', '+/-', 'season'], dtype=object)

In [ ]:
keep_col_trad = ['player', 'team', 'age', 'gp', 'min',
       'pts', 'fgm', 'fga', 'fg%', '3pm', '3pa', '3p%', 'ftm', 'fta',
       'ft%', 'oreb', 'ast', 'tov', '+/-', 'season']

keep_col_exceptions = ['player', 'team', 'age', 'gp', 'min', 'season']
speed_str = '_trad'

for i in range(110, 120):
    df = newfeature_rs_off[i]

    selected_cols = [col for col in keep_col_trad if col not in keep_col_exceptions]
    df = df[keep_col_trad]
    new_columns = [str(col) + speed_str if col in selected_cols else col for col in df.columns]
    df.columns = new_columns

    newfeature_rs_off[i] = df

---
## 1.2. Concatenación y unión de Dataframes Iniciales
---

In [ ]:
# Concatenar listas de dataframes OFENSIVO y DEFENSIVO

rs_concat_def = concatenate_dataframes(rs_def_df)
rs_concat_off = concatenate_dataframes(rs_off_df)

In [ ]:
len(rs_concat_def)

6

In [ ]:
rs_concat_def = name_dataframes(rs_concat_def, def_ls_names)

_speed
_def
_misc
_defim
_dreb
_adv


In [ ]:
rs_concat_off = name_dataframes(rs_concat_off, off_ls_names)

_adv
_misc
_scor
_szone
_cns
_dvs
_orb
_pass
_pup
_seff
_speed
_trad


In [ ]:
join_col_ls = ['season', 'player', 'season', 'min', 'team', 'gp', 'age']

In [ ]:
# Aplicación de función de join columns en toos los dataframes de la lista

def_df_precluster = []

for df in rs_concat_def:
  updated_df = keep_join_columns(df, join_col_ls)
  def_df_precluster.append(updated_df)

In [ ]:
for df in def_df_precluster:
  print(df.shape)

(5219, 14)
(5219, 22)
(5219, 21)
(5219, 13)
(5144, 15)
(5219, 25)


In [ ]:
clus_def_df = def_df_precluster[0]

# Iterate over the remaining DataFrames and merge them
for df in def_df_precluster[1:]:
    #Merge the DataFrames
    df_merged = pd.merge(clus_def_df, df, on = ['season', 'player', 'team'], how='inner', suffixes=('', '_drop'))


    #Drop the duplicate columns
    df_merged.drop([col for col in df_merged.columns if '_drop' in col], axis=1, inplace=True)
    # shift column 'Name' to first position
    first_column = df_merged.pop('season')

    # insert column using insert(position,column_name,
    # first_column) function
    df_merged.insert(0, 'season', first_column)

    # Update the master_df with the merged DataFrame
    clus_def_df = df_merged.sort_values(by=['player', 'season'])

In [ ]:
def_dup_col = find_duplicate_columns(clus_def_df)

print(def_dup_col)

['w_defim', 'l_misc', 'blk_misc', 'opp\xa0ptsoff\xa0to_misc', 'oppfbps_misc', 'opp2nd\xa0pts_misc', 'l_defim', 'defrtg_adv', 'unnamed: 0_adv', 'dreb%_adv', 'w_adv', 'w_misc', 'l_adv', 'opppitp_misc']


In [ ]:
clus_def_df = clus_def_df.drop(def_dup_col, axis=1)

In [ ]:
clus_def_df

,season,player,team,gp,w_speed,l_speed,min,dist. feet_speed,dist. miles_speed,dist. miles off_speed,...,ast ratio_adv,oreb%_adv,reb%_adv,to ratio_adv,efg%_adv,ts%_adv,usg%_adv,pace_adv,pie_adv,poss_adv
0,2022-23,A.J. Lawson,DAL,15,5,10,7.2,3155.2,0.60,0.32,...,3.8,4.6,9.1,5.8,61.4,58.9,18.9,105.96,7.6,240
1,2022-23,AJ Green,MIL,35,27,8,9.9,4053.2,0.77,0.42,...,14.0,1.6,6.1,5.7,60.0,60.7,15.9,102.90,7.6,744
2,2022-23,AJ Griffin,ATL,72,34,38,19.5,8386.3,1.59,0.85,...,10.9,2.6,5.3,6.3,56.0,57.7,17.4,102.67,7.7,3005
3,2016-17,AJ Hammons,DAL,22,4,18,7.4,2571.2,0.49,0.25,...,6.1,4.5,10.6,15.2,46.4,47.2,16.5,96.75,4.3,327
4,2013-14,AJ Price,MIN,28,15,13,3.5,1350.5,0.26,0.14,...,19.7,0.8,4.0,10.6,47.8,46.9,21.3,101.92,9.0,211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5117,2019-20,Zion Williamson,NOP,23,11,12,27.5,11307.0,2.14,1.13,...,9.1,9.2,10.4,10.8,59.2,61.6,29.1,107.01,14.1,1483
5118,2020-21,Zion Williamson,NOP,61,29,32,33.2,13804.7,2.61,1.41,...,13.7,8.1,10.6,10.1,61.6,64.9,28.7,103.30,16.1,4358
5119,2022-23,Zion Williamson,NOP,29,17,12,33.0,13026.6,2.47,1.34,...,16.5,6.0,10.4,12.3,61.5,65.2,28.8,102.17,17.1,2031
5120,2014-15,Zoran Dragic,MIA,16,6,10,4.7,1794.4,0.34,0.18,...,11.9,6.0,5.1,11.9,41.7,43.5,20.8,100.50,3.3,157


In [ ]:
# Filtrado de jugadores con pocos partidos para evitar ruido

clus_def_df = clus_def_df[clus_def_df['gp'] > 10]
print(clus_def_df.shape)

(4570, 69)


Ofensivo:

In [ ]:
# Aplicación de función de join columns en toos los dataframes de la lista

off_df_precluster = []

for df in rs_concat_off:
  updated_df = keep_join_columns(df, join_col_ls)
  off_df_precluster.append(updated_df)

In [ ]:
for df in off_df_precluster:
  # df['player'] = df['player'].apply(str.lower)
  df['player'] = df['player'].str.replace('Reggie Bullock Jr.', 'Reggie Bullock', regex=False)
  print(df.shape)

(5219, 25)
(5219, 21)
(5219, 24)
(5219, 25)
(5219, 13)
(5219, 24)
(5144, 15)
(5219, 16)
(5219, 15)
(5219, 21)
(5219, 14)
(5219, 31)


In [ ]:
clus_off_df = off_df_precluster[0]

# Iterate over the remaining DataFrames and merge them
for df in off_df_precluster[1:]:
    #Merge the DataFrames
    df_merged = pd.merge(clus_off_df, df, on = ['season', 'player', 'team'], how='inner', suffixes=('', '_drop'))


    #Drop the duplicate columns
    df_merged.drop([col for col in df_merged.columns if '_drop' in col], axis=1, inplace=True)
    # shift column 'Name' to first position
    first_column = df_merged.pop('season')

    # insert column using insert(position,column_name,
    # first_column) function
    df_merged.insert(0, 'season', first_column)

    # Update the master_df with the merged DataFrame
    clus_off_df = df_merged.sort_values(by=['player', 'season'])

In [ ]:
for df in off_df_precluster:
  problematic_row = df[df['player'] == 'jeenathan williams']
  # print(problematic_row)

In [ ]:
off_dup_col = find_duplicate_columns(clus_off_df)

print(off_dup_col)

['l_trad', 'l_scor', 'w_seff', 'w_speed', 'l_seff', 'l_speed', 'unnamed: 0_misc', 'blk_trad', 'drivefg%_seff', 'l_pup', 'pf_trad', 'unnamed: 0_scor', 'w_misc', 'l_pass', 'w_trad', 'w_scor', 'l_misc', 'w_pass', 'drivepts_seff', 'w_pup']


In [ ]:
clus_off_df = clus_off_df.drop(off_dup_col, axis=1)

In [ ]:
clus_off_df

,season,unnamed: 0_adv,player,team,age,gp,w_adv,l_adv,min,offrtg_adv,...,oreb_trad,dreb_trad,reb_trad,ast_trad,tov_trad,stl_trad,fp_trad,dd2_trad,td3_trad,+/-_trad
0,2022-23,451,A.J. Lawson,DAL,22,15,5,10,7.2,96.3,...,2.5,6.3,8.8,0.8,1.3,0.8,36.3,0.0,0.0,-19.2
1,2022-23,379,AJ Green,MIL,23,35,27,8,9.9,107.9,...,0.8,5.2,6.0,3.0,1.2,0.8,33.6,0.0,0.0,-3.5
2,2022-23,102,AJ Griffin,ATL,19,72,34,38,19.5,113.6,...,1.2,3.9,5.1,2.4,1.4,1.4,35.0,0.0,0.0,2.1
3,2016-17,403,AJ Hammons,DAL,24,22,4,18,7.4,101.2,...,2.4,8.6,11.0,1.2,3.1,0.3,39.5,0.0,0.0,-1.5
4,2013-14,376,AJ Price,MIN,27,28,15,13,3.5,89.1,...,0.5,4.3,4.7,6.2,3.3,0.5,33.9,0.0,0.0,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5127,2019-20,385,Zion Williamson,NOP,19,24,11,13,27.8,112.0,...,4.3,5.8,10.1,3.4,4.0,1.1,54.7,2.0,0.0,4.2
5128,2020-21,124,Zion Williamson,NOP,20,61,29,32,33.2,113.9,...,3.8,6.3,10.1,5.2,3.8,1.3,60.5,14.0,0.0,2.0
5129,2022-23,406,Zion Williamson,NOP,22,29,17,12,33.0,116.2,...,2.9,7.1,9.9,6.5,4.9,1.6,61.1,5.0,0.0,7.3
5130,2014-15,434,Zoran Dragic,MIA,26,16,6,10,4.7,92.4,...,3.2,1.9,5.1,3.2,3.2,1.3,29.4,0.0,0.0,-9.6


In [ ]:
# Filtrado de jugadores con pocos partidos para evitar ruido

clus_off_df = clus_off_df[clus_off_df['gp'] > 10]
print(clus_off_df.shape)

(4581, 167)


In [ ]:
# Revisión de diferencia entre los dataframes:

def_pl_seas = clus_def_df[['season', 'player', 'team']]
off_pl_seas = clus_off_df[['season', 'player', 'team']]

df_difference = pd.concat([def_pl_seas, off_pl_seas]).drop_duplicates(keep=False)

In [ ]:
df_difference

,season,player,team
2885,2014-15,Kenyon Martin,MIL
4096,2013-14,Reggie Bullock,LAC
4097,2014-15,Reggie Bullock,PHX
4098,2015-16,Reggie Bullock,DET
4099,2016-17,Reggie Bullock,DET
4100,2017-18,Reggie Bullock,DET
4101,2018-19,Reggie Bullock,LAL
4102,2019-20,Reggie Bullock,NYK
4103,2020-21,Reggie Bullock,NYK
4104,2021-22,Reggie Bullock,DAL


---
## 1.2.1. Concatenación y unión de Dataframes Extendidos
---

In [ ]:
# Concatenar listas de dataframes OFENSIVO y DEFENSIVO NBA

rs_concat_def_ext = concatenate_dataframes(rs_def_extended)
rs_concat_off_ext = concatenate_dataframes(rs_off_extended)

In [ ]:
# Concatenar listas de dataframes OFENSIVO y DEFENSIVO

rs_concat_def_bbref = concatenate_dataframes(rs_bbref_def)
rs_concat_off_bbref = concatenate_dataframes(rs_bbref_off)

In [ ]:
len(rs_concat_def_bbref)

2

In [ ]:
rs_concat_def_ext = name_dataframes(rs_concat_def_ext, def_ls_names_ext)
print('')
rs_concat_off_ext = name_dataframes(rs_concat_off_ext, off_ls_names_ext)

_def
_misc
_adv

_adv
_misc
_scor
_szone
_trad


In [ ]:
concat_def_bbref = name_dataframes(rs_concat_def_bbref, def_ls_bbref)
print('')
concat_off_bbref = name_dataframes(rs_concat_off_bbref, off_ls_bbref)

_bpm
_pbp

_bpm
_pbp
_shoot


In [ ]:
join_col_ls = ['season', 'player', 'min', 'team', 'gp', 'age']
join_col_ls_bbref = ['season', 'player', 'min']

In [ ]:
# Aplicación de función de join columns en toos los dataframes de la lista NBA

def_df_ext = []

for df in rs_concat_def_ext:
  updated_df = keep_join_columns(df, join_col_ls)
  updated_df = initial_name_cleaning(updated_df)
  def_df_ext.append(updated_df)

(12846, 22)
(12846, 21)
(12846, 25)


In [ ]:
clus_def_ext = def_df_ext[0]

# Iterate over the remaining DataFrames and merge them
for df in def_df_ext[1:]:
    #Merge the DataFrames
    df_merged = pd.merge(clus_def_ext, df, on = ['season', 'player', 'team'], how='inner', suffixes=('', '_drop'))


    #Drop the duplicate columns
    df_merged.drop([col for col in df_merged.columns if '_drop' in col], axis=1, inplace=True)
    # shift column 'Name' to first position
    first_column = df_merged.pop('season')

    # insert column using insert(position,column_name,
    # first_column) function
    df_merged.insert(0, 'season', first_column)

    # Update the master_df with the merged DataFrame
    clus_def_ext = df_merged.sort_values(by=['player', 'season'])

In [ ]:
def_dup_col = find_duplicate_columns(clus_def_ext)

print(def_dup_col)

['l_misc', 'blk_misc', 'opp\xa0ptsoff\xa0to_misc', 'oppfbps_misc', 'opp2nd\xa0pts_misc', 'w_misc', 'unnamed: 0_adv', 'dreb%_adv', 'w_adv', 'defrtg_adv', 'l_adv', 'opppitp_misc']


In [ ]:
clus_def_ext = clus_def_ext.drop(def_dup_col, axis=1)

In [ ]:
# Aplicación de función de join columns en toos los dataframes de la lista BBALLREF

def_df_bbref = []

for df in concat_def_bbref:
  updated_df = keep_join_columns(df, join_col_ls_bbref)
  updated_df = initial_name_cleaning(updated_df)
  def_df_bbref.append(updated_df)

(12839, 28)
(12839, 24)


In [ ]:
clus_def_ext_bbref = def_df_bbref[0]

# Iterate over the remaining DataFrames and merge them
for df in def_df_bbref[1:]:
    #Merge the DataFrames
    df_merged = pd.merge(clus_def_ext_bbref, df, on = ['season', 'player'], how='inner', suffixes=('', '_drop'))


    #Drop the duplicate columns
    df_merged.drop([col for col in df_merged.columns if '_drop' in col], axis=1, inplace=True)
    # shift column 'Name' to first position
    first_column = df_merged.pop('season')

    # insert column using insert(position,column_name,
    # first_column) function
    df_merged.insert(0, 'season', first_column)

    # Update the master_df with the merged DataFrame
    clus_def_ext_bbref = df_merged.sort_values(by=['player', 'season'])

In [ ]:
def_dup_col = find_duplicate_columns(clus_def_ext_bbref)

print(def_dup_col)

['position_pbp', 'g_pbp', 'tm_pbp', 'age_pbp', 'rk_pbp', 'min']


In [ ]:
clus_def_ext_bbref = clus_def_ext_bbref.drop(def_dup_col, axis=1)

In [ ]:
# Filtrado de jugadores con pocos partidos para evitar ruido NBA

clus_def_ext = clus_def_ext[clus_def_ext['gp'] > 10]
print(clus_def_ext.shape)

(11545, 44)


In [ ]:
clus_def_ext.columns.values

array(['season', 'unnamed: 0_def', 'player', 'team', 'age', 'gp', 'w_def',
       'l_def', 'min', 'def\xa0rtg_def', 'dreb_def', 'dreb%_def',
       '%dreb_def', 'stl_def', 'stl%_def', 'blk_def', '%blk_def',
       'opp\xa0ptsoff\xa0tov_def', 'opp\xa0pts2nd\xa0chance_def',
       'opp\xa0ptsfb_def', 'opp\xa0ptspaint_def', 'defws_def',
       'unnamed: 0_misc', 'ptsoff\xa0to_misc', '2ndpts_misc', 'fbps_misc',
       'pitp_misc', 'blka_misc', 'pf_misc', 'pfd_misc', 'offrtg_adv',
       'netrtg_adv', 'ast%_adv', 'ast/to_adv', 'ast\xa0ratio_adv',
       'oreb%_adv', 'reb%_adv', 'to\xa0ratio_adv', 'efg%_adv', 'ts%_adv',
       'usg%_adv', 'pace_adv', 'pie_adv', 'poss_adv'], dtype=object)

In [ ]:
keep_col_def = ['season', 'player', 'team', 'age', 'gp', 'min', 'def\xa0rtg_def', 'dreb_def', 'dreb%_def',
       '%dreb_def', 'stl_def', 'stl%_def', 'blk_def', '%blk_def',
       'opp\xa0ptsoff\xa0tov_def', 'opp\xa0pts2nd\xa0chance_def',
       'opp\xa0ptsfb_def', 'opp\xa0ptspaint_def', 'defws_def',
       'blka_misc', 'pf_misc', 'netrtg_adv', 'reb%_adv', 'pie_adv', 'poss_adv']

In [ ]:
clus_def_ext = clus_def_ext[keep_col_def]

In [ ]:
# Filtrado de jugadores con pocos partidos para evitar ruido BBALLREF
clus_def_ext_bbref['g_bpm'] = clus_def_ext_bbref['g_bpm'].astype(int)
clus_def_ext_bbref = clus_def_ext_bbref[clus_def_ext_bbref['g_bpm'] > 10]
print(clus_def_ext_bbref.shape)

(11552, 44)


In [ ]:
clus_def_ext_bbref.columns.values

array(['season', 'rk_bpm', 'player', 'pos_bpm', 'age_bpm', 'tm_bpm',
       'g_bpm', 'mp_bpm', 'per_bpm', 'ts%_bpm', '3par_bpm', 'ftr_bpm',
       'orb%_bpm', 'drb%_bpm', 'trb%_bpm', 'ast%_bpm', 'stl%_bpm',
       'blk%_bpm', 'tov%_bpm', 'usg%_bpm', 'ows_bpm', 'dws_bpm', 'ws_bpm',
       'ws/48_bpm', 'obpm_bpm', 'dbpm_bpm', 'bpm_bpm', 'vorp_bpm',
       'pg%_pbp', 'sg%_pbp', 'sf%_pbp', 'pf%_pbp', 'c%_pbp',
       '+/-_oncourt_pbp', '+/-_on-off_pbp', 'to_badpass_pbp',
       'to_lostball_pbp', 'fouls_committed_shoot_pbp',
       'fouls_committed_off_pbp', 'fouls_drawn_shoot_pbp',
       'fouls_drawn_off_pbp', 'pga_pbp', 'and1_pbp', 'blocked_pbp'],
      dtype=object)

In [ ]:
keep_col_def = ['season', 'player', 'pos_bpm', 'age_bpm', 'g_bpm', 'mp_bpm', 'per_bpm', 'drb%_bpm',
                'stl%_bpm', 'blk%_bpm', 'dws_bpm', 'ws_bpm', 'ws/48_bpm', 'dbpm_bpm', 'bpm_bpm', 'vorp_bpm',
                '+/-_oncourt_pbp', '+/-_on-off_pbp', 'fouls_committed_shoot_pbp',
                'fouls_drawn_off_pbp']

In [ ]:
clus_def_ext_bbref = clus_def_ext_bbref[keep_col_def]

In [ ]:
clus_def_ext = initial_name_cleaning(clus_def_ext)
clus_def_ext['season'] = clus_def_ext['season'].apply(convert_season_to_int)

clus_def_ext_bbref = initial_name_cleaning(clus_def_ext_bbref)

(11545, 25)
(11552, 20)


In [ ]:
def find_unmatched_players(df1, df2, on_columns):
    # Perform inner join on specified columns
    merged_df = pd.merge(df1, df2, on=on_columns, how='inner')

    # Get unique players from both DataFrames
    players_df1 = df1[on_columns[1]].unique()
    players_df2 = df2[on_columns[1]].unique()

    # Find players present in df1 but not in the merged_df
    unmatched_players = set(players_df1) - set(merged_df[on_columns[1]].unique())

    return list(unmatched_players)

In [ ]:
unmatched_players = find_unmatched_players(clus_def_ext_bbref, clus_def_ext, ['season', 'player'])
print(unmatched_players)
print(len(unmatched_players))

['Kiwane Lemorris Garris', 'Patrick Baldwin ', 'Perry Jones', 'Eugene Jeter', 'Glen Rice ', 'Harry Giles', 'Derrick Walton', 'Xavier Tillman Sr', 'Ha Seung-Jin', 'Melvin Frazier', 'Danny Schayes', "Boniface N'Dong", 'Kevin Knox', 'Clarence Weatherspoon', 'Mike Sweetney', 'Marcus Morris', 'Jeff Taylor', "Johnny O'Bryant", 'Reggie Bullock', 'Norm Richardson', 'Cameron Reynolds', 'Wade Baldwin', "Mamadou N'Diaye", 'Rich Manning', 'Isaac Austin', 'Steve Smith', 'John Butler', 'Vitor Luiz Faverani', 'Michael Frazier', 'RJ Nembhard ', 'Robert Williams', 'KJ Martin', 'Jeff Dowtin', 'Andrew White', 'Wang Zhizhi', "Hamady N'Diaye", 'Ronald Murray', 'Roger Mason', 'Stanislav Medvedenko']
39


In [ ]:
# Finding similar names in df2 using fuzzy matching
matched_names_dict = {}

for name in unmatched_players:
    match = process.extractOne(name, clus_def_ext['player'], scorer=fuzz.partial_ratio)
    if match[1] >= 80:  # Adjust the similarity threshold as needed
        matched_names_dict[name] = match[0]

In [ ]:
matched_names_dict

{'Perry Jones': 'Perry Jones I',
 'Glen Rice ': 'Glen Rice',
 'Harry Giles': 'Harry Giles I',
 'Derrick Walton': 'Derrick Walton ',
 'Xavier Tillman Sr': 'Xavier Tillman',
 'Melvin Frazier': 'Melvin Frazier ',
 'Danny Schayes': 'Dan Schayes',
 "Boniface N'Dong": 'Boniface Ndong',
 'Kevin Knox': 'Kevin Knox ',
 'Clarence Weatherspoon': 'Clar Weatherspoon',
 'Marcus Morris': 'Marcus Morris Sr',
 "Johnny O'Bryant": "Johnny O'Bryant I",
 'Reggie Bullock': 'Reggie Bullock ',
 'Norm Richardson': 'Jeremy Richardson',
 'Wade Baldwin': 'Wade Baldwin IV',
 "Mamadou N'Diaye": "Mamadou N'diaye",
 'Steve Smith': 'Steven Smith',
 'John Butler': 'John Butler ',
 'Michael Frazier': 'Michael Frazier ',
 'RJ Nembhard ': 'Andrew Nembhard',
 'Robert Williams': 'Robert Williams I',
 'KJ Martin': 'Darrick Martin',
 'Jeff Dowtin': 'Jeff Dowtin ',
 'Andrew White': 'Andrew White I',
 'Wang Zhizhi': 'Wang Zhi-zhi',
 "Hamady N'Diaye": 'Hamady Ndiaye',
 'Ronald Murray': 'Lamond Murray',
 'Roger Mason': 'Roger Mas

In [ ]:
matched_names_dict = {"Mamadou N'Diaye": "Mamadou N'diaye",
                      'Derrick Walton': 'Derrick Walton ',
                      'Steve Smith': 'Steven Smith',
                      'John Butler': 'John Butler ',
                      'Perry Jones': 'Perry Jones I',
                      'Michael Frazier': 'Michael Frazier ',
                      'Jeff Dowtin': 'Jeff Dowtin ',
                      'Wang Zhizhi': 'Wang Zhi-zhi',
                      'Xavier Tillman Sr': 'Xavier Tillman',
                      "Johnny O'Bryant": "Johnny O'Bryant I",
                      "Boniface N'Dong": 'Boniface Ndong',
                      'Stanislav Medvedenko': 'Slava Medvedenko',
                      'Harry Giles': 'Harry Giles I',
                      'Clarence Weatherspoon': 'Clar Weatherspoon',
                      "Hamady N'Diaye": 'Hamady Ndiaye',
                      'Melvin Frazier': 'Melvin Frazier ',
                      'Marcus Morris': 'Marcus Morris Sr',
                      'Glen Rice ': 'Glen Rice',
                      'Danny Schayes': 'Dan Schayes',
                      'Andrew White': 'Andrew White I',
                      'Kevin Knox': 'Kevin Knox ',
                      'Wade Baldwin': 'Wade Baldwin IV',
                      'Robert Williams': 'Robert Williams I',
                      'Roger Mason': 'Roger Mason ',
                      'Reggie Bullock': 'Reggie Bullock '}

matched_names_dict_flipped = {v: k for k, v in matched_names_dict.items()}

In [ ]:
clus_def_ext['player'] = clus_def_ext['player'].map(matched_names_dict_flipped).fillna(clus_def_ext['player'])

In [ ]:
# Unión de data NBA con BBALLREF

df_def_merged = pd.merge(clus_def_ext_bbref, clus_def_ext, on = ['season', 'player'], how='inner', suffixes=('', '_drop'))
df_def_merged = df_def_merged.sort_values(by=['season', 'player'])

In [ ]:
df_def_merged

,season,player,pos_bpm,age_bpm,g_bpm,mp_bpm,per_bpm,drb%_bpm,stl%_bpm,blk%_bpm,...,opp pts2nd chance_def,opp ptsfb_def,opp ptspaint_def,defws_def,blka_misc,pf_misc,netrtg_adv,reb%_adv,pie_adv,poss_adv
0,1997,AC Green,PF,33,83,2492,12.4,20.5,1.5,0.5,...,14.9,15.9,44.5,0.142,0.9,3.1,-7.4,13.5,9.8,4699
49,1997,Aaron McKie,SG,24,83,1625,12.1,13.2,2.6,1.1,...,13.8,10.7,34.4,0.268,1.2,4.2,3.7,7.1,9.5,3084
65,1997,Aaron Williams,PF-C,25,33,563,16.4,16.9,1.5,3.9,...,17.6,15.0,49.2,0.180,0.7,6.8,-9.3,12.9,11.2,1062
82,1997,Acie Earl,C,26,47,500,10.7,14.7,1.6,4.5,...,13.6,18.3,46.0,0.219,0.8,6.2,-6.4,9.4,5.6,987
88,1997,Adam Keefe,C,26,62,915,11.7,18.2,1.7,1.2,...,13.4,10.8,38.2,0.229,0.7,5.5,7.2,12.9,8.3,1766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11419,2023,Zach Collins,C,25,63,1441,16.4,22.5,1.2,2.8,...,15.0,14.3,54.5,0.093,1.1,6.4,-7.5,13.3,11.0,3121
11428,2023,Zach LaVine,SG,27,77,2768,19.0,12.2,1.2,0.6,...,11.4,13.0,47.1,0.158,1.4,2.8,0.3,6.4,12.6,5750
11471,2023,Zeke Nnaji,PF,22,53,728,12.5,11.2,1.1,2.8,...,13.4,13.2,53.1,0.219,0.8,7.1,-5.9,9.3,7.2,1488
11483,2023,Ziaire Williams,SF,21,37,561,7.4,11.9,1.2,1.0,...,15.6,14.1,45.7,0.150,0.4,4.9,-5.2,6.7,5.2,1195


In [ ]:
df_def_merged.to_csv(base_folder_string + 'def_extended_for_feat_engin.csv')

Ofensivo:

In [ ]:
# Aplicación de función de join columns en toos los dataframes de la lista NBA

off_df_ext = []

for df in rs_concat_off_ext:
  updated_df = keep_join_columns(df, join_col_ls)
  updated_df = initial_name_cleaning(updated_df)
  off_df_ext.append(updated_df)

(12846, 25)
(12846, 21)
(12846, 24)
(12846, 25)
(12846, 31)


In [ ]:
clus_off_ext = off_df_ext[0]

# Iterate over the remaining DataFrames and merge them
for df in off_df_ext[1:]:
    #Merge the DataFrames
    df_merged = pd.merge(clus_off_ext, df, on = ['season', 'player'], how='inner', suffixes=('', '_drop'))


    #Drop the duplicate columns
    df_merged.drop([col for col in df_merged.columns if '_drop' in col], axis=1, inplace=True)
    # shift column 'Name' to first position
    first_column = df_merged.pop('season')

    # insert column using insert(position,column_name,
    # first_column) function
    df_merged.insert(0, 'season', first_column)

    # Update the master_df with the merged DataFrame
    clus_off_ext = df_merged.sort_values(by=['player', 'season'])

In [ ]:
clus_off_ext.columns.values

array(['season', 'unnamed: 0_adv', 'player', 'team', 'age', 'gp', 'w_adv',
       'l_adv', 'min', 'offrtg_adv', 'defrtg_adv', 'netrtg_adv',
       'ast%_adv', 'ast/to_adv', 'ast\xa0ratio_adv', 'oreb%_adv',
       'dreb%_adv', 'reb%_adv', 'to\xa0ratio_adv', 'efg%_adv', 'ts%_adv',
       'usg%_adv', 'pace_adv', 'pie_adv', 'poss_adv', 'unnamed: 0_misc',
       'w_misc', 'l_misc', 'ptsoff\xa0to_misc', '2ndpts_misc',
       'fbps_misc', 'pitp_misc', 'opp\xa0ptsoff\xa0to_misc',
       'opp2nd\xa0pts_misc', 'oppfbps_misc', 'opppitp_misc', 'blk_misc',
       'blka_misc', 'pf_misc', 'pfd_misc', 'unnamed: 0_scor', 'w_scor',
       'l_scor', '%fga2pt_scor', '%fga3pt_scor', '%pts2pt_scor',
       '%pts2pt\xa0mr_scor', '%pts3pt_scor', '%ptsfbps_scor',
       '%ptsft_scor', '%ptsoffto_scor', '%ptspitp_scor', '2fgm%ast_scor',
       '2fgm%uast_scor', '3fgm%ast_scor', '3fgm%uast_scor',
       'fgm%ast_scor', 'fgm%uast_scor', 'ra_fgm_szone', 'ra_fga_szone',
       'ra_fg%_szone', 'paint_fgm_szone', 'pa

In [ ]:
keep_col_off = ['season', 'player', 'team', 'gp', 'min', 'offrtg_adv',
       'netrtg_adv', 'ast%_adv', 'ast/to_adv', 'ast\xa0ratio_adv',
       'oreb%_adv', 'to\xa0ratio_adv', 'efg%_adv', 'ts%_adv', 'usg%_adv',
       'pace_adv', 'pie_adv', 'poss_adv', 'ptsoff\xa0to_misc',
       '2ndpts_misc', 'fbps_misc', 'pitp_misc', 'pfd_misc',
       '%fga2pt_scor', '%fga3pt_scor', '%pts2pt_scor',
       '%pts2pt\xa0mr_scor', '%pts3pt_scor', '%ptsfbps_scor',
       '%ptsft_scor', '%ptsoffto_scor', '%ptspitp_scor', '2fgm%ast_scor',
       '2fgm%uast_scor', '3fgm%ast_scor', '3fgm%uast_scor',
       'fgm%ast_scor', 'fgm%uast_scor', 'ra_fgm_szone', 'ra_fga_szone',
       'ra_fg%_szone', 'paint_fgm_szone', 'paint_fga_szone',
       'paint_fg%_szone', 'mr_fgm_szone', 'mr_fga_szone', 'mr_fg%_szone',
       'lc3_fgm_szone', 'lc3_fga_szone', 'lc3_fg%_szone', 'rc3_fgm_szone',
       'rc3_fga_szone', 'rc3_fg%_szone', 'c3_fga_szone', 'c3_fga.1_szone',
       'c3_fga.2_szone', 'ab3_fgm_szone', 'ab3_fga_szone',
       'ab3_fg%_szone', 'pts_trad', 'fgm_trad',
       'fga_trad', 'fg%_trad', '3pm_trad', '3pa_trad', '3p%_trad',
       'ftm_trad', 'fta_trad', 'ft%_trad', 'oreb_trad', 'ast_trad', 'tov_trad',
       '+/-_trad']

In [ ]:
clus_off_ext = clus_off_ext[keep_col_off]

In [ ]:
off_dup_col = find_duplicate_columns(clus_off_ext)

print(off_dup_col)

[]


In [ ]:
clus_off_ext = clus_off_ext.drop(off_dup_col, axis=1)

In [ ]:
# Filtrado de jugadores con pocos partidos para evitar ruido

clus_off_ext = clus_off_ext[clus_off_ext['gp'] > 10]
clus_off_ext['season'] = clus_off_ext['season'].apply(convert_season_to_int)

print(clus_off_ext.shape)

(11602, 73)


In [ ]:
# Aplicación de función de join columns en toos los dataframes de la lista BBALLREF

off_df_bbref = []

for df in concat_off_bbref:
  updated_df = keep_join_columns(df, join_col_ls_bbref)
  updated_df = initial_name_cleaning(updated_df)
  off_df_bbref.append(updated_df)

(12839, 28)
(12839, 24)
(12839, 36)


In [ ]:
_bpm
_pbp
_shoot

In [ ]:
clus_off_bbref = off_df_bbref[0]

# Iterate over the remaining DataFrames and merge them
for df in off_df_bbref[1:]:
    #Merge the DataFrames
    df_merged = pd.merge(clus_off_bbref, df, on = ['season', 'player'], how='inner', suffixes=('', '_drop'))


    #Drop the duplicate columns
    df_merged.drop([col for col in df_merged.columns if '_drop' in col], axis=1, inplace=True)
    # shift column 'Name' to first position
    first_column = df_merged.pop('season')

    # insert column using insert(position,column_name,
    # first_column) function
    df_merged.insert(0, 'season', first_column)

    # Update the master_df with the merged DataFrame
    clus_off_bbref = df_merged.sort_values(by=['player', 'season'])

In [ ]:
clus_off_bbref.columns.values

array(['season', 'rk_bpm', 'player', 'pos_bpm', 'age_bpm', 'tm_bpm',
       'g_bpm', 'mp_bpm', 'per_bpm', 'ts%_bpm', '3par_bpm', 'ftr_bpm',
       'orb%_bpm', 'drb%_bpm', 'trb%_bpm', 'ast%_bpm', 'stl%_bpm',
       'blk%_bpm', 'tov%_bpm', 'usg%_bpm', 'ows_bpm', 'dws_bpm', 'ws_bpm',
       'ws/48_bpm', 'obpm_bpm', 'dbpm_bpm', 'bpm_bpm', 'vorp_bpm',
       'rk_pbp', 'position_pbp', 'age_pbp', 'tm_pbp', 'g_pbp', 'min',
       'pg%_pbp', 'sg%_pbp', 'sf%_pbp', 'pf%_pbp', 'c%_pbp',
       '+/-_oncourt_pbp', '+/-_on-off_pbp', 'to_badpass_pbp',
       'to_lostball_pbp', 'fouls_committed_shoot_pbp',
       'fouls_committed_off_pbp', 'fouls_drawn_shoot_pbp',
       'fouls_drawn_off_pbp', 'pga_pbp', 'and1_pbp', 'blocked_pbp',
       'rk_shoot', 'position_shoot', 'age_shoot', 'tm_shoot', 'g_shoot',
       'fg%_shoot', 'dist_shoot', 'unnamed_9_level_1_shoot',
       '%_fga_by_distance_2p_shoot', '%_fga_by_distance_0-3_shoot',
       '%_fga_by_distance_3-10_shoot', '%_fga_by_distance_10-16_shoot',
  

In [ ]:
keep_col_off = ['season', 'player', 'pos_bpm', 'age_bpm',
       'g_bpm', 'mp_bpm', 'per_bpm', 'ts%_bpm', '3par_bpm', 'ftr_bpm',
       'orb%_bpm', 'ast%_bpm', 'tov%_bpm', 'usg%_bpm', 'ows_bpm', 'ws_bpm',
       'ws/48_bpm', 'obpm_bpm', 'bpm_bpm', 'vorp_bpm',
       '+/-_oncourt_pbp', '+/-_on-off_pbp', 'to_badpass_pbp',
       'to_lostball_pbp',
       'fouls_committed_off_pbp', 'fouls_drawn_shoot_pbp',
       'pga_pbp', 'and1_pbp', 'blocked_pbp',
       'fg%_shoot', 'dist_shoot',
       '%_fga_by_distance_2p_shoot', '%_fga_by_distance_0-3_shoot',
       '%_fga_by_distance_3-10_shoot', '%_fga_by_distance_10-16_shoot',
       '%_fga_by_distance_16-3p_shoot', '%_fga_by_distance_3p_shoot',
       'fg%_2p_shoot', 'fg%_0-3_shoot',
       'fg%_3-10_shoot', 'fg%_10-16_shoot', 'fg%_16-3p_shoot',
       'fg%_3p_shoot', '%_fg_astd_2p_shoot',
       '%_fg_astd_3p_shoot',
       'dunks_%fga_shoot', 'dunks_#_shoot',
       'corner_3s_%3pa_shoot', 'corner_3s_3p%_shoot',
       'heaves_att_shoot']

In [ ]:
clus_off_bbref = clus_off_bbref[keep_col_off]

In [ ]:
off_dup_col = find_duplicate_columns(clus_off_bbref)

print(off_dup_col)

['%_fga_by_distance_3p_shoot']


In [ ]:
clus_off_bbref = clus_off_bbref.drop(off_dup_col, axis=1)

In [ ]:
# Filtrado de jugadores con pocos partidos para evitar ruido

clus_off_bbref['g_bpm'] = clus_off_bbref['g_bpm'].astype(int)
clus_off_bbref = clus_off_bbref[clus_off_bbref['g_bpm'] > 10]
print(clus_off_bbref.shape)

(11552, 49)


In [ ]:
clus_off_ext['player'] = clus_off_ext['player'].map(matched_names_dict_flipped).fillna(clus_off_ext['player'])

In [ ]:
# Unión de data NBA con BBALLREF

df_off_merged = pd.merge(clus_off_bbref, clus_off_ext, on = ['season', 'player'], how='inner', suffixes=('', '_drop'))
df_off_merged = df_off_merged.sort_values(by=['season', 'player'])

In [ ]:
df_off_merged

,season,player,pos_bpm,age_bpm,g_bpm,mp_bpm,per_bpm,ts%_bpm,3par_bpm,ftr_bpm,...,3pm_trad,3pa_trad,3p%_trad,ftm_trad,fta_trad,ft%_trad,oreb_trad,ast_trad,tov_trad,+/-_trad
0,1997,AC Green,PF,33,83,2492,12.4,.523,.041,.407,...,0.0,0.4,5.0,2.7,4.2,65.0,4.7,1.5,1.6,-7.4
49,1997,Aaron McKie,SG,24,83,1625,12.1,.524,.282,.301,...,1.3,3.3,39.8,3.0,3.6,83.6,1.3,5.2,2.9,4.3
65,1997,Aaron Williams,PF-C,25,33,563,16.4,.599,.007,.331,...,0.0,0.1,0.0,3.1,4.6,67.3,5.8,1.4,3.0,-9.8
82,1997,Acie Earl,C,26,47,500,10.7,.433,.028,.467,...,0.0,0.5,0.0,5.5,8.5,64.3,3.5,2.0,3.5,-6.1
88,1997,Adam Keefe,C,26,62,915,11.7,.572,.006,.644,...,0.0,0.1,0.0,4.0,5.8,68.9,4.2,1.8,2.5,6.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,2023,Zach Collins,C,25,63,1441,16.4,.599,.268,.259,...,1.8,4.7,37.4,3.5,4.5,76.1,3.7,5.8,4.1,-7.7
11455,2023,Zach LaVine,SG,27,77,2768,19.0,.607,.392,.308,...,3.5,9.5,37.5,6.3,7.4,84.8,0.7,5.7,3.4,0.3
11498,2023,Zeke Nnaji,PF,22,53,728,12.5,.620,.332,.316,...,1.1,4.4,26.2,2.7,4.2,64.5,4.4,1.2,2.1,-6.5
11510,2023,Ziaire Williams,SF,21,37,561,7.4,.511,.495,.112,...,2.1,8.1,25.8,1.4,1.8,77.3,1.3,2.9,3.1,-6.4


In [ ]:
df_off_merged.to_csv(base_folder_string + 'off_extended_for_feat_engin.csv')

---
# 2. Filtrado de columnas en dataframe DEFENSIVO
---

In [ ]:
drop_col_def = ['ptsoff to_misc', '2ndpts_misc', 'fbps_misc',
                'pitp_misc', 'dist. feet_speed', 'dist. miles off_speed', 'avg speed off_speed',
                'unnamed: 0_def', 'w_dreb', 'l_dreb']

In [ ]:
clus_def_df = clus_def_df.drop(drop_col_def, axis=1)

In [ ]:
clus_def_df

,season,player,team,gp,w_speed,l_speed,min,dist. miles_speed,dist. miles def_speed,avg speed_speed,...,ast ratio_adv,oreb%_adv,reb%_adv,to ratio_adv,efg%_adv,ts%_adv,usg%_adv,pace_adv,pie_adv,poss_adv
0,2022-23,A.J. Lawson,DAL,15,5,10,7.2,0.60,0.28,4.64,...,3.8,4.6,9.1,5.8,61.4,58.9,18.9,105.96,7.6,240
1,2022-23,AJ Green,MIL,35,27,8,9.9,0.77,0.34,4.35,...,14.0,1.6,6.1,5.7,60.0,60.7,15.9,102.90,7.6,744
2,2022-23,AJ Griffin,ATL,72,34,38,19.5,1.59,0.74,4.56,...,10.9,2.6,5.3,6.3,56.0,57.7,17.4,102.67,7.7,3005
3,2016-17,AJ Hammons,DAL,22,4,18,7.4,0.49,0.23,3.94,...,6.1,4.5,10.6,15.2,46.4,47.2,16.5,96.75,4.3,327
4,2013-14,AJ Price,MIN,28,15,13,3.5,0.26,0.11,4.37,...,19.7,0.8,4.0,10.6,47.8,46.9,21.3,101.92,9.0,211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5138,2022-23,Ziaire Williams,MEM,37,21,16,15.2,1.15,0.57,4.22,...,12.6,2.8,6.7,13.4,49.2,51.1,17.8,102.80,5.2,1195
5139,2019-20,Zion Williamson,NOP,23,11,12,27.5,2.14,1.01,4.39,...,9.1,9.2,10.4,10.8,59.2,61.6,29.1,107.01,14.1,1483
5140,2020-21,Zion Williamson,NOP,61,29,32,33.2,2.61,1.20,4.42,...,13.7,8.1,10.6,10.1,61.6,64.9,28.7,103.30,16.1,4358
5141,2022-23,Zion Williamson,NOP,29,17,12,33.0,2.47,1.12,4.19,...,16.5,6.0,10.4,12.3,61.5,65.2,28.8,102.17,17.1,2031


In [ ]:
# Exportación de dataframe para revisar columnas y que no falte mucha info.

clus_def_df.to_csv(base_folder_string + 'defense_data_2014_2023.csv')

---
# 3. Filtrado de columnas en dataframe OFENSIVO
---

In [ ]:
drop_col_off = ['defrtg_adv', 'dreb%_adv', 'reb%_adv', 'opp ptsoff to_misc',
                'opp2nd pts_misc', 'oppfbps_misc', 'opppitp_misc', 'blka_misc',
                'pfd_misc', 'dist. feet_speed',
                'dist. miles def_speed', 'avg speed def_speed', 'unnamed: 0_trad',
                'dreb_trad', 'reb_trad', 'stl_trad', 'fp_trad', 'poss_adv', 'w_orb', 'l_orb']

In [ ]:
clus_off_df = clus_off_df.drop(drop_col_off, axis=1)

In [ ]:
clus_off_df.columns

Index(['season', 'unnamed: 0_adv', 'player', 'team', 'age', 'gp', 'w_adv',
       'l_adv', 'min', 'offrtg_adv',
       ...
       '3p%_trad', 'ftm_trad', 'fta_trad', 'ft%_trad', 'oreb_trad', 'ast_trad',
       'tov_trad', 'dd2_trad', 'td3_trad', '+/-_trad'],
      dtype='object', length=147)

In [ ]:
# Exportación de dataframe para revisar columnas y que no falte mucha info.

clus_off_df.to_csv(base_folder_string + 'offense_data_2014_2023.csv')

---
# 4. Importar CSV y preparar datos de BPM Reg.Season
---

In [ ]:
regseas_bpm_string = '/content/drive/MyDrive/Colab Notebooks/TFM/bpm/reg_season_{}.csv'

In [ ]:
rs_bpm_df = get_bpm_dataframes(increased_season_bpm, regseas_bpm_string)

/content/drive/MyDrive/Colab Notebooks/TFM/bpm/reg_season_{}.csv


In [ ]:
rs_bpm_df[5].sort_values(by="VORP", ascending=False)

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,season
77,65,Vince Carter,SF,24,TOR,75,2979,25.0,0.551,0.240,...,10.3,2.6,12.9,0.208,NaN,7.6,0.1,7.6,7.2,2001
377,295,Shaquille O'Neal*,C,28,LAL,74,2924,30.2,0.574,0.001,...,11.1,3.9,14.9,0.245,NaN,7.0,0.7,7.7,7.1,2001
317,254,Tracy McGrady*,SG,21,ORL,77,3087,24.9,0.521,0.096,...,8.3,3.9,12.2,0.189,NaN,6.5,0.5,7.0,7.0,2001
173,138,Kevin Garnett*,PF,24,MIN,81,3202,23.9,0.531,0.045,...,7.0,4.8,11.8,0.176,NaN,4.5,1.4,6.0,6.4,2001
296,236,Karl Malone*,PF,37,UTA,81,2895,24.7,0.572,0.004,...,9.1,4.0,13.1,0.217,NaN,5.5,1.1,6.6,6.3,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,278,Hanno Möttölä,SF,24,ATL,73,989,7.5,0.504,0.011,...,-0.1,0.5,0.4,0.018,NaN,-5.1,-1.5,-6.6,-1.1,2001
21,20,Vin Baker,PF,29,SEA,76,2129,13.0,0.481,0.019,...,0.6,1.2,1.8,0.040,NaN,-2.4,-1.8,-4.2,-1.2,2001
160,127,Marcus Fizer,PF,22,CHI,72,1580,11.0,0.476,0.060,...,-1.3,0.6,-0.7,-0.022,NaN,-3.3,-2.8,-6.1,-1.6,2001
9,8,John Amaechi,C,30,ORL,82,1710,8.7,0.455,0.012,...,-1.3,1.4,0.0,0.001,NaN,-4.3,-1.8,-6.1,-1.7,2001


In [ ]:
rs_bpm_df[9].columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'season'],
      dtype='object')

In [ ]:
# Concatenar lista de dataframes de BPM

rs_bpm_concat_df = concatenate_dataframes(rs_bpm_df)

In [ ]:
rs_bpm_concat_df = rs_bpm_concat_df[0]

In [ ]:
rs_bpm_concat_df.sort_values(by=['season', 'player'])

,rk,player,pos,age,tm,g,mp,per,ts%,3par,...,ows,dws,ws,ws/48,unnamed: 24,obpm,dbpm,bpm,vorp,season
0,147,A.C. Green,SF,32,PHO,82,2113,12.9,0.558,0.117,...,2.7,1.7,4.4,0.100,NaN,-0.4,-0.9,-1.3,0.4,1996
1,244,Aaron McKie,SG,23,POR,81,2259,13.4,0.534,0.162,...,2.1,3.2,5.3,0.113,NaN,-1.3,1.7,0.4,1.4,1996
2,107,Acie Earl,C,25,TOR,42,655,14.1,0.484,0.011,...,-0.1,0.4,0.3,0.020,NaN,-3.1,-2.0,-5.1,-0.5,1996
3,192,Adam Keefe,PF,25,UTA,82,1708,13.8,0.574,0.012,...,2.3,2.3,4.6,0.130,NaN,-0.5,0.0,-0.6,0.6,1996
4,58,Adrian Caldwell,PF,29,IND,51,327,10.4,0.556,0.000,...,-0.2,0.5,0.3,0.044,NaN,-5.1,-0.7,-5.8,-0.3,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13262,97,Zach Collins,C,25,SAS,63,1441,16.4,0.599,0.268,...,1.2,1.0,2.2,0.074,NaN,-0.5,0.3,-0.2,0.6,2023
13263,283,Zach LaVine,SG,27,CHI,77,2768,19.0,0.607,0.392,...,4.2,2.9,7.1,0.123,NaN,2.7,-0.7,1.9,2.7,2023
13264,362,Zeke Nnaji,PF,22,DEN,53,728,12.5,0.620,0.332,...,0.9,0.7,1.6,0.106,NaN,-2.7,-0.9,-3.7,-0.3,2023
13265,526,Ziaire Williams,SF,21,MEM,37,561,7.4,0.511,0.495,...,-0.5,0.6,0.1,0.011,NaN,-4.7,-0.8,-5.6,-0.5,2023


In [ ]:
sel_bpm_columns = ['player', 'season', 'g', 'mp', 'obpm', 'dbpm', 'bpm', 'vorp']

In [ ]:
rs_bpm_concat_df = rs_bpm_concat_df[sel_bpm_columns]
rs_bpm_concat_df = rs_bpm_concat_df.sort_values(by=['season', 'player'], ascending=True)

In [ ]:
rs_bpm_concat_df

,player,season,g,mp,obpm,dbpm,bpm,vorp
0,A.C. Green,1996,82,2113,-0.4,-0.9,-1.3,0.4
1,Aaron McKie,1996,81,2259,-1.3,1.7,0.4,1.4
2,Acie Earl,1996,42,655,-3.1,-2.0,-5.1,-0.5
3,Adam Keefe,1996,82,1708,-0.5,0.0,-0.6,0.6
4,Adrian Caldwell,1996,51,327,-5.1,-0.7,-5.8,-0.3
...,...,...,...,...,...,...,...,...
13262,Zach Collins,2023,63,1441,-0.5,0.3,-0.2,0.6
13263,Zach LaVine,2023,77,2768,2.7,-0.7,1.9,2.7
13264,Zeke Nnaji,2023,53,728,-2.7,-0.9,-3.7,-0.3
13265,Ziaire Williams,2023,37,561,-4.7,-0.8,-5.6,-0.5


In [ ]:
obpm_q99 = rs_bpm_concat_df.obpm.quantile(0.99)
obpm_q1 = rs_bpm_concat_df.obpm.quantile(0.01)
off_outliers_df = rs_bpm_concat_df[(rs_bpm_concat_df['obpm'] > obpm_q99) | (rs_bpm_concat_df['obpm'] < obpm_q1)]

In [ ]:
# Gráfico de dispersión comparativo

# Create the scatter plot using Plotly Express
fig = px.scatter(off_outliers_df, x='obpm', y='g', color='mp',
                 hover_data=['player'])

# Customize the layout if needed
fig.update_layout(title='Gráfico de dispersión OBPM vs PARTIDOS JUGADOS',
                  xaxis_title='OBPM TEMPORADA REGULAR',
                  yaxis_title='PARTIDOS JUGADOS',
                  hovermode='closest')

fig.update_traces(marker=dict(size=8))

# Show the interactive plot
fig.show()

In [ ]:
dbpm_q99 = rs_bpm_concat_df.dbpm.quantile(0.99)
dbpm_q1 = rs_bpm_concat_df.dbpm.quantile(0.01)
def_outliers_df = rs_bpm_concat_df[(rs_bpm_concat_df['dbpm'] > dbpm_q99) | (rs_bpm_concat_df['dbpm'] < dbpm_q1)]

In [ ]:
# Gráfico de dispersión comparativo

# Create the scatter plot using Plotly Express
fig = px.scatter(def_outliers_df, x='dbpm', y='g', color='mp',
                 hover_data=['player'])

# Customize the layout if needed
fig.update_layout(title='Comparación en DBPM vs PARTIDOS JUGADOS',
                  xaxis_title='DBPM TEMPORADA REGULAR',
                  yaxis_title='PARTIDOS JUGADOS',
                  hovermode='closest')

fig.update_traces(marker=dict(size=8))

# Show the interactive plot
fig.show()

In [ ]:
'''
min_gp = 15
rs_bpm_concat_df = rs_bpm_concat_df[rs_bpm_concat_df['g'] >= min_gp]
'''

"\nmin_gp = 15\nrs_bpm_concat_df = rs_bpm_concat_df[rs_bpm_concat_df['g'] >= min_gp]\n"

---
# 4.1. Importar CSV y preparar datos de BPM Playoffs
---

In [ ]:
playoff_bpm_string = '/content/drive/MyDrive/Colab Notebooks/TFM/playoff_bpm_data_bballref/playoffs_{}.csv'

In [ ]:
playoff_bpm_df = get_bpm_dataframes(increased_season_bpm, playoff_bpm_string)

In [ ]:
playoff_bpm_df[4].sort_values(by="VORP", ascending=False)

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,season
91,88,Allen Iverson*,SG,25,PHI,22,1016,22.5,.480,.215,...,1.9,0.9,2.7,.130,NaN,5.5,0.6,6.1,2.1,2001
0,1,Ray Allen*,SG,25,MIL,18,768,23.7,.613,.360,...,3.2,0.4,3.6,.223,NaN,8.2,0.1,8.3,2.0,2001
22,22,Kobe Bryant*,SG,22,LAL,16,694,25.0,.555,.095,...,2.6,1.2,3.8,.260,NaN,6.0,0.6,6.5,1.5,2001
133,128,Shaquille O'Neal*,C,28,LAL,16,676,28.7,.564,.000,...,2.3,1.4,3.7,.260,NaN,6.9,-0.5,6.5,1.4,2001
127,122,Dikembe Mutombo*,C,34,PHI,23,981,19.7,.584,.005,...,2.4,1.3,3.8,.184,NaN,2.1,1.0,3.1,1.3,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,54,Howard Eisley,SG,28,DAL,9,194,6.5,.475,.491,...,-0.2,0.0,-0.1,-0.032,NaN,-2.6,-0.4,-3.1,-0.1,2001
136,131,Greg Ostertag,C,27,UTA,5,64,3.7,.313,.000,...,-0.1,0.0,-0.1,-0.041,NaN,-4.4,-1.4,-5.8,-0.1,2001
166,160,Reggie Slater,PF,30,MIN,4,52,5.7,.385,.000,...,-0.1,0.1,0.0,-0.001,NaN,-7.5,0.0,-7.6,-0.1,2001
29,29,Elden Campbell,PF,32,CHH,10,287,14.2,.474,.000,...,0.1,0.5,0.6,.100,NaN,-2.5,-0.8,-3.3,-0.1,2001


In [ ]:
# Concatenar lista de dataframes de BPM

playoff_bpm_concat_df = concatenate_dataframes(playoff_bpm_df)

In [ ]:
playoff_bpm_concat_df = playoff_bpm_concat_df[0]

In [ ]:
sel_bpm_columns = ['player', 'season', 'g', 'mp', 'obpm', 'dbpm', 'bpm', 'vorp']

In [ ]:
playoff_bpm_concat_df = playoff_bpm_concat_df[sel_bpm_columns]
playoff_bpm_concat_df = playoff_bpm_concat_df.sort_values(by=['season', 'player'], ascending=True)

In [ ]:
new_col_names = ['player', 'season', 'g_playoffs', 'mp_playoffs', 'obpm_playoffs', 'dbpm_playoffs', 'bpm_playoffs', 'vorp_playoffs']
playoff_bpm_concat_df.columns = new_col_names

In [ ]:
playoff_bpm_concat_df.tail(3)

,player,season,g_playoffs,mp_playoffs,obpm_playoffs,dbpm_playoffs,bpm_playoffs,vorp_playoffs
5433,Yuta Watanabe,2023,1,5,-6.3,-8.9,-15.1,0.0
5434,Zeke Nnaji,2023,5,12,-0.2,2.4,2.2,0.0
5435,Ziaire Williams,2023,4,12,-6.3,-1.9,-8.2,0.0


In [ ]:
playoff_bpm_concat_df = initial_name_cleaning(playoff_bpm_concat_df)

(5436, 8)


---
# 5. Unión de datos BPM Playoffs con otra info para aprendizaje supervizado
---

In [ ]:
# Importar archivos de OFF y DEF

def_raw_df = pd.read_csv(base_folder_string + 'def_extended_for_feat_engin.csv', low_memory=False)
off_raw_df = pd.read_csv(base_folder_string + 'off_extended_for_feat_engin.csv', low_memory=False)

In [ ]:
print('Def raw: ', def_raw_df.shape)
print('POFF bpm: ', playoff_bpm_concat_df.shape)

Def raw:  (11503, 44)
POFF bpm:  (5436, 8)


In [ ]:
# Unión de data NBA con BBALLREF

df_def_bpm_merged = pd.merge(def_raw_df, playoff_bpm_concat_df, on = ['season', 'player'], how='inner', suffixes=('', '_drop'))
df_def_bpm_merged = df_def_bpm_merged.sort_values(by=['season', 'player'])
df_def_bpm_merged.shape

(5292, 50)

In [ ]:
df_def_bpm_merged = df_def_bpm_merged.drop('obpm_playoffs', axis=1)

In [ ]:
unmatched_players = find_unmatched_players(playoff_bpm_concat_df, def_raw_df, ['season', 'player'])
print(len(unmatched_players))

51


In [ ]:
# Assuming 'def_raw_df' and 'playoff_bpm_concat_df' are your DataFrames
merged_df = pd.merge(playoff_bpm_concat_df, def_raw_df, on=['season', 'player'], how='left', indicator=True)

unmatched_players = merged_df[merged_df['_merge'] == 'left_only'][['season', 'player', 'mp_playoffs']]

In [ ]:
unmatched_players

,season,player,mp_playoffs
10,1997,Bison Dele,336
65,1997,Glen Rice,137
78,1997,Isaac Austin,287
125,1997,Mike Brown,27
142,1997,Rich Manning,21
...,...,...,...
5380,2023,Omer Yurtseven,16
5385,2023,Patrick Baldwin,11
5407,2023,Shaquille Harrison,28
5421,2023,Tristan Thompson,32


Estos jugadores "Perdidos" se generan porque no jugaron más de 10 partidos en temp. regular pero si algo en playoffs.

In [ ]:
# Exportación de dataframe DEFENSIVO.

df_def_bpm_merged.to_csv(base_folder_string + 'defense_ml_1997_2023.csv')

In [ ]:
# Unión de data NBA con BBALLREF

df_off_bpm_merged = pd.merge(off_raw_df, playoff_bpm_concat_df, on = ['season', 'player'], how='inner', suffixes=('', '_drop'))
df_off_bpm_merged = df_off_bpm_merged.sort_values(by=['season', 'player'])
df_off_bpm_merged.shape

(5291, 127)

In [ ]:
# Exportación de dataframe DEFENSIVO.
df_off_bpm_merged = df_off_bpm_merged.drop('dbpm_playoffs', axis=1)
df_off_bpm_merged.to_csv(base_folder_string + 'offense_ml_1997_2023.csv')

---
# 5.1. Unión de datos BPM RegSeason Playoffs
---

In [ ]:
print(rs_bpm_concat_df.shape)
print(playoff_bpm_concat_df.shape)

(13267, 8)
(5624, 8)


In [ ]:
rs_bpm_concat_df = initial_name_cleaning(rs_bpm_concat_df)

playoff_bpm_concat_df = initial_name_cleaning(playoff_bpm_concat_df)

(13267, 8)
(5624, 8)


In [ ]:
print(rs_bpm_concat_df.tail(3))
print('')
print(playoff_bpm_concat_df.tail(3))

                player  season   g   mp  obpm  dbpm  bpm  vorp
13264       Zeke Nnaji    2023  53  728  -2.7  -0.9 -3.7  -0.3
13265  Ziaire Williams    2023  37  561  -4.7  -0.8 -5.6  -0.5
13266  Zion Williamson    2023  29  956   4.8   1.0  5.8   1.9

               player  season  g  mp  obpm  dbpm   bpm  vorp
5612    Yuta Watanabe    2023  1   5  -6.3  -8.9 -15.1   0.0
5557       Zeke Nnaji    2023  5  12  -0.2   2.4   2.2   0.0
5619  Ziaire Williams    2023  4  12  -6.3  -1.9  -8.2   0.0


In [ ]:
# Unión de Dataframes BPM para comparación

bpm_comp_df = pd.merge(rs_bpm_concat_df, playoff_bpm_concat_df, on = ['player', 'season'], how='inner', suffixes=('_regseas', '_playoffs'))

# bpm_comp_df.sort_values(by='obpm_playoffs', ascending=False).head(10)

bpm_comp_df

,player,season,g_regseas,mp_regseas,obpm_regseas,dbpm_regseas,bpm_regseas,vorp_regseas,g_playoffs,mp_playoffs,obpm_playoffs,dbpm_playoffs,bpm_playoffs,vorp_playoffs
0,AC Green,1996,82,2113,-0.4,-0.9,-1.3,0.4,4,87,-2.5,-2.0,-4.5,-0.1
1,Aaron McKie,1996,81,2259,-1.3,1.7,0.4,1.4,5,134,-3.7,2.4,-1.3,0.0
2,Adam Keefe,1996,82,1708,-0.5,0.0,-0.6,0.6,17,178,-1.2,0.6,-0.6,0.1
3,Adrian Caldwell,1996,51,327,-5.1,-0.7,-5.8,-0.3,1,3,14.2,2.3,16.5,0.0
4,Alan Henderson,1996,79,1416,-3.2,-0.9,-4.1,-0.8,10,145,-1.3,-1.2,-2.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5609,Wesley Matthews,2023,52,820,-3.4,0.8,-2.7,-0.1,2,41,1.6,3.2,4.8,0.1
5610,Xavier Tillman Sr,2023,61,1180,0.3,1.7,1.9,1.2,6,183,0.1,3.1,3.2,0.2
5611,Yuta Watanabe,2023,58,928,-0.4,-0.2,-0.6,0.3,1,5,-6.3,-8.9,-15.1,0.0
5612,Zeke Nnaji,2023,53,728,-2.7,-0.9,-3.7,-0.3,5,12,-0.2,2.4,2.2,0.0


In [ ]:
bpm_comp_df_playerseason = bpm_comp_df[['player', 'season']]
playoff_bpm_concat_df_playerseason = playoff_bpm_concat_df[['player', 'season']]

In [ ]:
bpm_diff = pd.concat([playoff_bpm_concat_df_playerseason, bpm_comp_df_playerseason]).drop_duplicates(keep=False)

In [ ]:
# List of player names from your first DataFrame
player_names = bpm_diff['player'].tolist()

# Filter the rows in other_df where the 'player' column matches any name from the list
filtered_df = rs_bpm_concat_df[rs_bpm_concat_df['player'].isin(player_names)]

In [ ]:
bpm_diff

,player,season
396,Mike Brown,1998
3377,Dwayne Jones,2013
3403,Tracy McGrady,2013
4092,Dorell Wright,2016
3963,John Holland,2016
4421,Ty Lawson,2018
4734,Jaylen Adams,2020
5390,Luca Vildoza,2022
5503,DaQuan Jeffries,2023
5601,Tristan Thompson,2023


In [ ]:
# Exportación de dataframe DEFENSIVO.

bpm_comp_df.to_csv(base_folder_string + 'bpm_comparison_data.csv')

---
# 6. Unión de dataframes filtrados por temporada
---

DEFENSIVO:

In [ ]:
len(newfeature_rs_def)

60

In [ ]:
# Concatenar listas de dataframes DEFENSIVO

rs_newfeature_def = concatenate_dataframes(newfeature_rs_def)

In [ ]:
for df in rs_newfeature_def:
  print(df.shape)

(4591, 6)
(4592, 18)
(4592, 12)
(4591, 8)
(4336, 13)
(4592, 9)


In [ ]:
clus_newfeat_def_df = rs_newfeature_def[0]

# Iterate over the remaining DataFrames and merge them
for df in rs_newfeature_def[1:]:
    #Merge the DataFrames
    df_merged = pd.merge(clus_newfeat_def_df, df, on = ['season', 'player', 'team'], how='inner', suffixes=('', '_drop'))
    first_column = df_merged.pop('season')

    # insert column using insert(position,column_name,
    # first_column) function
    df_merged.insert(0, 'season', first_column)

    # Update the master_df with the merged DataFrame
    clus_newfeat_def_df = df_merged.sort_values(by=['player', 'season'])

In [ ]:
clus_newfeat_def_df.shape

(4336, 51)

In [ ]:
clus_def_ls = find_duplicate_columns(clus_newfeat_def_df)

In [ ]:
clus_def_ls

['opp2nd\xa0pts_misc',
 'blk_misc',
 'dreb%_defadv',
 'gp_drop',
 'oppfbps_misc',
 'opp\xa0ptsoff\xa0to_misc',
 'opppitp_misc',
 'min_drop',
 'defrtg_defadv']

In [ ]:
clus_newfeat_def_df.drop(clus_def_ls, axis=1, inplace=True)

In [ ]:
clus_newfeat_def_df.columns.values

array(['season', 'player', 'team', 'gp', 'min',
       'dist.\xa0miles\xa0def_speed', 'def\xa0rtg_def', 'dreb_def',
       'dreb%_def', '%dreb_def', 'stl_def', 'stl%_def', 'blk_def',
       '%blk_def', 'opp\xa0ptsoff\xa0tov_def',
       'opp\xa0pts2nd\xa0chance_def', 'opp\xa0ptsfb_def',
       'opp\xa0ptspaint_def', 'defws_def', 'blka_misc', 'pf_misc',
       'dfgm_imp', 'dfga_imp', 'dfg%_imp', 'dreb_reb',
       'contesteddreb_reb', 'contesteddreb%_reb', 'drebchances_reb',
       'drebchance%_reb', 'deferreddreb\xa0chances_reb',
       'adjusteddreb\xa0chance%_reb', 'avg\xa0drebdistance_reb',
       'pie_defadv', 'poss_defadv'], dtype=object)

In [ ]:
clus_newfeat_def_df['season'] = clus_newfeat_def_df['season'].apply(convert_season_to_int)

In [ ]:
bpm_comp_df.columns.values

array(['player', 'season', 'g', 'mp_regseas', 'obpm_regseas',
       'dbpm_regseas', 'bpm_regseas', 'vorp_regseas', 'mp_playoffs',
       'obpm_playoffs', 'dbpm_playoffs', 'bpm_playoffs', 'vorp_playoffs'],
      dtype=object)

In [ ]:
def_bpm_col = ['player', 'season', 'dbpm_regseas', 'bpm_regseas', 'vorp_regseas', 'dbpm_playoffs']

def_bpm = bpm_comp_df[def_bpm_col]
def_bpm = def_bpm.copy()
def_bpm['season'] = def_bpm['season'].astype(int)

In [ ]:
def_bpm.sort_values(by=['player', 'season'])

,player,season,dbpm_regseas,bpm_regseas,vorp_regseas,dbpm_playoffs
1533,Aaron Brooks,2015,-1.1,-0.8,0.5,-2.2
1532,Aaron Gordon,2019,0.3,0.6,1.7,1.8
1531,Aaron Gordon,2021,-0.1,0.1,0.7,-1.8
1530,Aaron Gordon,2022,-1.1,-0.6,0.9,-2.2
1529,Aaron Gordon,2023,-0.3,2.1,2.1,-0.6
...,...,...,...,...,...,...
8,Zach Randolph,2017,-1.6,-0.5,0.7,-2.3
7,Zaza Pachulia,2015,0.8,0.4,1.0,2.8
6,Zaza Pachulia,2016,0.1,-0.2,0.9,2.0
5,Zaza Pachulia,2017,1.6,0.6,0.8,0.7


In [ ]:
playername_dic = generate_name_dictionary(clus_newfeat_def_df, def_bpm)

In [ ]:
clus_newfeat_def_df['player'] = clus_newfeat_def_df['player'].apply(lambda x: standardize_player_name(x, playername_dic))
def_bpm['player'] = def_bpm['player'].apply(lambda x: standardize_player_name(x, playername_dic))

In [ ]:
def_newfeat_merged = pd.merge(clus_newfeat_def_df, def_bpm, on = ['season', 'player'], how='inner', suffixes=('', '_drop'))

In [ ]:
def_newfeat_merged.shape # Falta organizar la función de nombres para que no se pierdan jugadores

# Debería haber 2114 filas...

(1534, 38)

In [ ]:
def_newfeat_merged.sort_values(by='vorp_regseas', ascending=False).head()

,season,player,team,gp,min,dist. miles def_speed,def rtg_def,dreb_def,dreb%_def,%dreb_def,...,drebchance%_reb,deferreddreb chances_reb,adjusteddreb chance%_reb,avg drebdistance_reb,pie_defadv,poss_defadv,dbpm_regseas,bpm_regseas,vorp_regseas,dbpm_playoffs
1128,2022,Nikola Jokic,DEN,74,33.5,1.05,108.9,15.8,31.3,42.9,...,70.4,1.2,76.3,4.0,23.0,5136,4.5,13.7,9.8,2.5
835,2014,Kevin Durant,OKC,81,38.5,1.20,103.3,8.5,16.4,24.2,...,72.2,0.4,75.5,6.9,20.6,6342,1.4,10.2,9.6,0.4
1358,2016,Stephen Curry,GSW,79,34.2,1.13,99.8,6.3,11.7,17.1,...,55.6,0.6,59.8,9.3,19.7,5779,1.6,11.9,9.5,0.4
632,2019,James Harden,HOU,78,36.8,1.06,110.3,7.7,15.7,24.4,...,64.4,0.7,70.0,6.9,20.1,5851,1.6,11.0,9.3,1.7
1302,2017,Russell Westbrook,OKC,81,34.6,1.09,106.1,12.4,25.1,35.2,...,80.2,0.4,82.8,6.1,23.0,5854,2.4,11.1,9.3,2.2


In [ ]:
def_newfeat_merged.to_csv(base_folder_string + 'defense_feature_analysis.csv')

OFENSIVO:

In [ ]:
# Concatenar listas de dataframes DEFENSIVO

rs_newfeature_off = concatenate_dataframes(newfeature_rs_off)

In [ ]:
for df in rs_newfeature_off:
  print(df.shape)

(5219, 16)
(5219, 10)
(5219, 20)
(5219, 24)
(5219, 13)
(5219, 22)
(5144, 13)
(5219, 13)
(5219, 12)
(5219, 17)
(5219, 7)
(5219, 20)


In [ ]:
clus_newfeat_off_df = rs_newfeature_off[0]

# Iterate over the remaining DataFrames and merge them
for df in rs_newfeature_off[1:]:
    #Merge the DataFrames
    df_merged = pd.merge(clus_newfeat_off_df, df, on = ['season', 'player', 'team'], how='inner', suffixes=('', '_drop'))
    first_column = df_merged.pop('season')

    # insert column using insert(position,column_name,
    # first_column) function
    df_merged.insert(0, 'season', first_column)

    # Update the master_df with the merged DataFrame
    clus_newfeat_off_df = df_merged.sort_values(by=['player', 'season'])

In [ ]:
clus_newfeat_off_df.shape # Aquí debería ser 5144 jugadores

(5133, 154)

In [ ]:
clus_off_ls = find_duplicate_columns(clus_newfeat_off_df)

In [ ]:
clus_off_ls

['drivepts_shooting_eff',
 'age_drop',
 'gp_drop',
 'min_drop',
 'drivefg%_shooting_eff']

In [ ]:
clus_newfeat_off_df.drop(clus_off_ls, axis=1, inplace=True)

In [ ]:
clus_newfeat_off_df['season'] = clus_newfeat_off_df['season'].apply(convert_season_to_int)

In [ ]:
off_bpm_col = ['player', 'season', 'obpm_regseas', 'obpm_regseas', 'vorp_regseas', 'obpm_playoffs']

off_bpm = bpm_comp_df[off_bpm_col]
off_bpm = off_bpm.copy()
off_bpm['season'] = off_bpm['season'].astype(int)

In [ ]:
off_bpm.sort_values(by=['player', 'season'])

,player,season,obpm_regseas,obpm_regseas,vorp_regseas,obpm_playoffs
1533,Aaron Brooks,2015,0.3,0.3,0.5,-4.4
1532,Aaron Gordon,2019,0.3,0.3,1.7,1.7
1531,Aaron Gordon,2021,0.2,0.2,0.7,-2.2
1530,Aaron Gordon,2022,0.5,0.5,0.9,0.9
1529,Aaron Gordon,2023,2.4,2.4,2.1,0.4
...,...,...,...,...,...,...
8,Zach Randolph,2017,1.0,1.0,0.7,-1.6
7,Zaza Pachulia,2015,-0.4,-0.4,1.0,-1.6
6,Zaza Pachulia,2016,-0.3,-0.3,0.9,2.4
5,Zaza Pachulia,2017,-1.1,-1.1,0.8,-2.5


In [ ]:
playername_dic = generate_name_dictionary(clus_newfeat_off_df, off_bpm)

In [ ]:
clus_newfeat_off_df['player'] = clus_newfeat_off_df['player'].apply(lambda x: standardize_player_name(x, playername_dic))
off_bpm['player'] = off_bpm['player'].apply(lambda x: standardize_player_name(x, playername_dic))

In [ ]:
off_newfeat_merged = pd.merge(clus_newfeat_off_df, off_bpm, on = ['season', 'player'], how='inner', suffixes=('', '_drop'))

In [ ]:
off_newfeat_merged.shape # Falta organizar la función de nombres para que no se pierdan jugadores

# Debería haber 2114 filas...

(1532, 136)

In [ ]:
off_newfeat_merged.to_csv(base_folder_string + 'offense_feature_analysis.csv')

---
# 6.1. Unión de dataframes para análisis de ML con temp. extendidas
---

In [ ]:
'''
clus_def_ext
clus_off_ext
bpm_comp_df
'''

'\nclus_def_ext\nclus_off_ext\nbpm_comp_df\n'

In [ ]:
clus_def_ext = initial_name_cleaning(clus_def_ext)
clus_def_ext['season'] = clus_def_ext['season'].apply(convert_season_to_int)

clus_off_ext = initial_name_cleaning(clus_off_ext)
clus_off_ext['season'] = clus_off_ext['season'].apply(convert_season_to_int)

bpm_comp_df = initial_name_cleaning(bpm_comp_df)

(11539, 25)
(12825, 80)
(5614, 14)


In [ ]:
bpm_comp_df.sort_values(by=['season', 'vorp_regseas'], ascending=False)

,player,season,g_regseas,mp_regseas,obpm_regseas,dbpm_regseas,bpm_regseas,vorp_regseas,g_playoffs,mp_playoffs,obpm_playoffs,dbpm_playoffs,bpm_playoffs,vorp_playoffs
5555,Nikola Jokic,2023,69,2323,8.5,4.5,13.0,8.8,20,789,9.1,3.7,12.8,3.0
5500,Joel Embiid,2023,66,2284,6.8,2.3,9.2,6.4,9,336,0.9,1.0,1.9,0.3
5496,Jimmy Butler,2023,64,2138,6.7,2.0,8.7,5.8,22,874,5.0,1.5,6.5,1.9
5453,Domantas Sabonis,2023,79,2736,4.1,1.8,5.8,5.4,7,243,0.1,0.7,0.7,0.2
5466,Giannis Antetokounmpo,2023,63,2024,5.8,2.7,8.5,5.4,3,91,1.3,0.6,2.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,Alan Henderson,1996,79,1416,-3.2,-0.9,-4.1,-0.8,10,145,-1.3,-1.2,-2.4,0.0
77,Felton Spencer,1996,71,1267,-4.3,-0.2,-4.5,-0.8,18,276,-6.9,1.0,-5.9,-0.3
104,Joe Wolf,1996,64,1065,-4.2,-0.6,-4.8,-0.8,11,85,-8.8,-2.2,-11.0,-0.2
20,Bobby Hurley,1996,72,1059,-4.1,-1.8,-5.9,-1.0,1,2,-5.2,-2.6,-7.9,0.0


In [ ]:
# Lista de nombres DEFENSA
def_diff_ls = pd.concat([clus_def_ext, bpm_comp_df]).drop_duplicates(keep=False)
player_names_def = def_diff_ls['player'].tolist()
print(len(player_names_def))

# Lista de nombres ATAQUE
off_diff_ls = pd.concat([clus_off_ext, bpm_comp_df]).drop_duplicates(keep=False)
player_names_off = off_diff_ls['player'].tolist()
print(len(player_names_off))

17153
18439


In [ ]:
def group_similar_names(player_names, threshold=85):
    similar_names = defaultdict(list)

    for name in player_names:
        added = False
        for group, grouped_names in similar_names.items():
            for grouped_name in grouped_names:
                if (fuzz.ratio(name, grouped_name) >= threshold or
                        fuzz.partial_ratio(name, grouped_name) >= threshold):
                    similar_names[group].append(name)
                    added = True
                    break
            if added:
                break
        if not added:
            similar_names[name].append(name)

    standardized_names = {}
    for group, names in similar_names.items():
        standardized_names[choose_standard_name(names)] = names

    return standardized_names

In [ ]:
'''
similar_names_result = group_similar_names(player_names_off)
'''

In [ ]:
json_file_path = base_folder_string + 'player_name_dic.json'

with open(json_file_path, 'r') as j:
     similar_names_result = json.loads(j.read())

In [ ]:
with open(base_folder_string + 'player_name_dic.json', 'w') as file:
    json.dump(similar_names_result, file)

In [ ]:
'''
clus_def_ext['player'] = clus_def_ext['player'].apply(lambda x: standardize_player_name(x, similar_names_result))
clus_off_ext['player'] = clus_off_ext['player'].apply(lambda x: standardize_player_name(x, similar_names_result))
bpm_comp_df['player'] = bpm_comp_df['player'].apply(lambda x: standardize_player_name(x, similar_names_result))
'''

In [ ]:
def_extended_merged = pd.merge(clus_def_ext, bpm_comp_df, on = ['player', 'season'], how='inner', suffixes=('', '_drop'))

In [ ]:
def_extended_merged.sort_values(by='dbpm_regseas', ascending=False)

,season,player,team,age,gp,min,def rtg_def,dreb_def,dreb%_def,%dreb_def,...,obpm_regseas,dbpm_regseas,bpm_regseas,vorp_regseas,g_playoffs,mp_playoffs,obpm_playoffs,dbpm_playoffs,bpm_playoffs,vorp_playoffs
1906,2012,Greg Stiemsma,BOS,26,55,52.3,96.5,8.8,16.2,26.5,...,-4.0,5.5,1.4,0.7,19,143,-2.0,4.6,2.6,0.2
3222,2018,Lucas Nogueira,TOR,25,49,47.9,96.2,7.9,16.0,24.3,...,0.1,5.0,5.2,0.8,5,24,-13.0,-2.2,-15.2,-0.1
2000,2022,Isaac Bonga,TOR,22,15,45.5,106.7,2.0,3.8,6.7,...,-5.4,5.0,-0.4,0.0,1,3,-0.1,4.4,4.3,0.0
1470,2022,Draymond Green,GSW,32,46,47.5,102.8,10.4,20.4,28.5,...,-1.4,4.6,3.2,1.8,22,703,-2.0,2.9,0.9,0.5
3794,2022,Nikola Jokic,DEN,27,74,48.2,108.9,15.8,31.3,42.9,...,9.2,4.5,13.7,9.8,5,171,8.1,2.5,10.7,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5074,2002,Victor Alexander,DET,32,14,49.0,96.9,11.3,21.6,31.9,...,-5.3,-3.5,-8.8,-0.2,1,3,-10.8,-3.9,-14.7,0.0
239,2019,Anfernee Simons,POR,20,20,45.5,122.1,3.2,6.7,10.2,...,-1.5,-3.7,-5.3,-0.1,5,12,-14.2,-4.2,-18.5,-0.1
436,2008,Billy Thomas,CLE,32,11,49.2,110.0,2.4,5.3,8.0,...,-8.6,-3.7,-12.3,-0.1,3,8,9.8,8.8,18.6,0.0
2962,2002,Kirk Haston,CHH,23,15,46.3,93.4,9.0,17.0,24.2,...,-7.5,-3.8,-11.3,-0.2,2,4,5.8,13.8,19.5,0.0


In [ ]:
off_extended_merged = pd.merge(clus_off_ext, bpm_comp_df, on = ['season', 'player'], how='inner', suffixes=('', '_drop'))

In [ ]:
off_extended_merged.sort_values(by='obpm_regseas', ascending=False)

,season,player,team,gp,min,offrtg_adv,netrtg_adv,ast%_adv,ast/to_adv,ast ratio_adv,...,obpm_regseas,dbpm_regseas,bpm_regseas,vorp_regseas,g_playoffs,mp_playoffs,obpm_playoffs,dbpm_playoffs,bpm_playoffs,vorp_playoffs
4706,2016,Stephen Curry,GSW,79,34.2,117.5,17.7,31.5,2.01,20.7,...,10.3,1.6,11.9,9.5,18,614,6.6,0.4,7.0,1.4
5035,2003,Tracy McGrady,ORL,75,39.3,108.1,3.6,28.2,2.11,15.0,...,9.8,0.7,10.5,9.3,7,308,9.6,1.1,10.6,1.0
3209,2009,LeBron James,CLE,81,37.7,114.0,14.6,36.5,2.44,21.2,...,9.5,3.7,13.2,11.8,14,580,12.8,4.8,17.5,2.9
2234,2019,James Harden,HOU,78,36.8,116.6,6.3,39.4,1.51,18.2,...,9.4,1.6,11.0,9.3,11,424,6.4,1.7,8.1,1.1
3213,2013,LeBron James,MIA,76,37.9,114.2,12.3,34.4,2.44,23.3,...,9.3,2.4,11.7,9.9,23,960,8.4,2.0,10.4,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4454,2004,Ryan Humphrey,MEM,2,5.8,156.5,98.2,8.3,0.50,14.3,...,-12.2,-2.3,-14.6,0.0,3,5,13.4,-1.3,12.1,0.0
454,2012,Blake Ahearn,UTA,4,7.6,82.5,-22.4,6.3,0.20,5.0,...,-12.7,-6.8,-19.5,-0.1,3,8,17.3,3.7,21.0,0.0
812,2018,Chinanu Onuaku,HOU,1,21.6,80.0,-17.6,10.0,0.33,11.1,...,-13.0,-1.7,-14.7,-0.1,1,3,4.7,17.3,21.9,0.0
3069,2021,Kris Dunn,ATL,4,11.4,86.3,-31.4,8.0,0.67,10.5,...,-13.7,-0.1,-13.7,-0.1,5,33,-6.7,4.9,-1.8,0.0


In [ ]:
drop_col_def = ['obpm_regseas', 'obpm_playoffs', 'mp_playoffs', 'bpm_playoffs', 'vorp_playoffs']
drop_col_off = ['dbpm_regseas', 'dbpm_playoffs', 'mp_playoffs', 'bpm_playoffs', 'vorp_playoffs']

In [ ]:
def_extended_merged = def_extended_merged.drop(drop_col_def, axis=1)
off_extended_merged = off_extended_merged.drop(drop_col_off, axis=1)

In [ ]:
def_extended_merged.to_csv(base_folder_string + 'def_ext_feature_analysis.csv')
off_extended_merged.to_csv(base_folder_string + 'off_ext_feature_analysis.csv')

In [ ]:
off_def_ext = pd.merge(off_extended_merged, def_extended_merged, on = ['season', 'player'], how='inner', suffixes=('', '_drop'))

In [ ]:
dup_ls = find_duplicate_columns(off_def_ext)

In [ ]:
off_def_ext = off_def_ext.drop(dup_ls, axis=1)

In [ ]:
off_def_ext.to_csv(base_folder_string + 'all_data_feature_analysis.csv')